In [1]:
import pandas as pd
import os
import numpy as np
import csv
import tldextract
import matplotlib.pyplot as plt
import seaborn as sns
import re

import email.header
import email.utils
import string
import sys
from dataclasses import dataclass, field
from ietfdata.datatracker import *
from ietfdata.mailarchive import *
import ietfdata.mailarchive as ma
import datetime
from datetime import datetime
from datetime import date
import json
import tldextract
%config InlineBackend.figure_format = 'retina'

In [10]:
import networkx as nx
from networkx.algorithms import centrality

In [177]:
import sys
print (sys.version)

3.9.4 (default, Apr  9 2021, 09:32:38) 
[Clang 10.0.0 ]


To generate the data structures used in this work: emailID_pid_dict.json, pid_emailID_dict.json, name_pid_dict.json, role_based_emailIDs_newData.txt , automated_email_IDs_newData.txt please refer to https://github.com/sodestream/imc2021-submission/blob/main/scripts/emails/email-messages-year.py and https://github.com/sodestream/imc2021-submission

In [2]:
ren = r'(?:\.?)([\w\-_+#~!$&\'\.]+(?<!\.)(@|[ ]?\(?[ ]?(at|AT)[ ]?\)?[ ]?)(?<!\.)[\w]+[\w\-\.]*\.[a-zA-Z-]{2,3})(?:[^\w])'
ren2 = r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)'

In [3]:
emailID_pid_dict = json.load(open('../analysis_revision3/emailID_pid_dict.json'))
pid_emailID_dict = json.load(open('../analysis_revision3/pid_emailID_dict.json'))
name_pid_dict = json.load(open('../analysis_revision3/name_pid_dict.json'))

In [4]:
role_based_emailIDs = set()
with open('../analysis_revision3/role_based_emailIDs_newData.txt') as f:
    Lines = f.readlines()
    for line in Lines:
        line = line.strip()
        role_based_emailIDs.add(line)
        #print(line)
print(len(role_based_emailIDs))

856


In [75]:
#role_based_emailIDs

In [5]:
automated_list = set()
with open('../analysis_revision3/automated_email_IDs_newData.txt') as f2:
    Lines2 = f2.readlines()
    for line in Lines2:
        line = line.strip()
        automated_list.add(line)
        #print(line)
print(len(automated_list))

685


To generate interaction mappings please refer to https://github.com/sodestream/icwsm22/blob/master/V2-generate_personID_personID_mappings.py and https://github.com/sodestream/icwsm22

In [6]:
#if the file is compressed and zipped, please unzip
df_mappings = pd.read_csv('../analysis_revision3/personID_to_personID_graph_new_with_mlisttype.csv',delimiter='\t',header=0)

df_mappings['Interaction_timestamp'] = pd.to_datetime(df_mappings['Interaction_timestamp'], format='%Y-%m-%d').dt.date
df_mappings.head(3)

,A_PersonID_From,B_PersonID_To,Type,Time_since_1st_mail_A,Time_since_1st_mail_B,Max_Time_A,Max_Time_B,Interaction_timestamp,MessageID_A,MessageID_B,Mailing_list,Mailinglist_type
0,100005,100001,reply_to,0.0,6.5,0.0,9.5,2017-11-13,<CAKFu26gHc2mutnyNhRMz9YekRb60angmBiWgz5Z6sUDC...,<1262915113.716847.1510540285719@mail.yahoo.com>,100-newcomers,other
1,100001,100001,reply_self,6.5,6.5,9.5,9.5,2017-11-14,<277920060.1462457.1510625963022@mail.yahoo.com>,<1262915113.716847.1510540285719@mail.yahoo.com>,100-newcomers,other
2,100006,100002,reply_to,0.0,15.0,1.5,18.5,2017-11-16,<CADVGGb9CHep2dQY9P+E7ODaOEkhm7bTjk7OWKDQz0VWK...,<5FE99815-6CC8-4FF2-B7FB-709D0BEEEF5E@amsl.com>,100-newcomers,other


For a reference to spam messages scoring please contact Mladen Karan: m.karan at qmul (dot) ac (dot) uk

In [7]:
spam_messageIDs = set([])

for file in os.listdir("../spamres/"):
    if file.endswith(".csv"):
        fname = os.path.join("../spamres/", file)
        fdf = pd.read_csv(fname)
        #print(file)
        for i, r in fdf.iterrows():
            
            if r['header_flag'] == r['header_flag'] and 'yes' in r['header_flag'].lower():
                spam_messageIDs.add(r['mid'])
            elif r['SA_score'] >= 6.5:
                spam_messageIDs.add(r['mid'])
            
        print(file,len(spam_messageIDs))

slim.csv 3
rohc.csv 8
datatracker-rqmts.csv 8
it-test.csv 8
lemonade.csv 13
tsv-area.csv 14
rescap.csv 24
ieprep.csv 27
websec.csv 35
108all.csv 35
perc.csv 35
sshmgmt.csv 35
mobopts.csv 35
ospf-wireless-design.csv 35
ccamp.csv 907
nsis.csv 946
hybi.csv 952
svt.csv 952
secretariat-2012.csv 952
tzdist.csv 952
ipdvb.csv 961
69attendees.csv 964
96attendees.csv 964
100-newcomers.csv 964
tip.csv 967
llc-consultation.csv 967
opsec.csv 973
104all.csv 973
stox.csv 973
91attendees.csv 974
dcrouting.csv 974
certid.csv 974
new-wg-docs.csv 974
mib-doctors.csv 974
channel-binding.csv 974
bmwg.csv 1021
art.csv 1022
v3.csv 1022
ipae.csv 1022
sami.csv 1025
ietf-meetings.csv 1025
nasreq.csv 1031
nmlrg.csv 1031
endymail.csv 1032
oda.csv 1032
geojson.csv 1032
84attendees.csv 1034
l2cp.csv 1034
collation.csv 1034
dhcp.csv 1034
hostmib.csv 1034
multipathtcp.csv 1038
rsn.csv 1038
104-newcomers.csv 1038
isn.csv 1041
mcic.csv 1041
rfcplusplus.csv 1041
spud.csv 1041
laminar.csv 1041
regext.csv 1060
cose.csv 10

opsawg.csv 6759
ml_testlist.csv 6759
video-codec.csv 6759
web.csv 6761
uri.csv 6763
104attendees.csv 6763
eat.csv 6763
cidrd.csv 6764
remoteui.csv 6764
perpass.csv 6765
imrg.csv 6766
103attendees.csv 6766
l2sm.csv 6766
pwe3.csv 6769
mhtml.csv 6769
conneg.csv 6774
emu.csv 6779
rai-discuss.csv 6779
ietf108planning.csv 6779
84all.csv 6779
entmib.csv 6784
mavs.csv 6784
ieee-ietf-coord.csv 6784
bgp.csv 6784
arcing.csv 6786
mpvdapi.csv 6790
precis.csv 6790
testlist.csv 6791
siesta.csv 6791
vpn4dc.csv 6791
marnew.csv 6791
njm.csv 6791
rats.csv 6793
big-internet.csv 6793
ident.csv 6793
ipsra.csv 6959
iesg-agenda-dist.csv 6959
vot.csv 6959
hip.csv 6959
ipsp.csv 7351
96all.csv 7351
ibnemo.csv 7352
actn.csv 7355
3gv6.csv 7364
89attendees.csv 7370
resolverless-dns.csv 7370
76attendees.csv 7370
diffserv.csv 7380
sip.csv 7469
sipping-emergency.csv 7469
stackevo-discuss.csv 7469
yaco-wgchair-tracker.csv 7469
822ext.csv 7469
jwt-reg-review.csv 7469
lsvr.csv 7469
wgchairs.csv 7509
71attendees.csv 7509


rmcat.csv 15552
dispatch.csv 15563
sixpac.csv 15563
scuba-ag.csv 15563
106-newcomers.csv 15563
plus.csv 15563
shim6.csv 15565
spwg.csv 15565
101companions.csv 15565
xml2rfc.csv 15625
wpack.csv 15625
caris-attendees.csv 15625
homenet.csv 15625
aggsrv.csv 15625
tcpsat.csv 15626
ntp.csv 15638
bfcpbis.csv 15638
ldap-dir.csv 16112
addr-select-dt.csv 16112
catnip.csv 16112
renum.csv 16112
ipngwg.csv 16358
ipr-announce.csv 16358
crypto-panel.csv 16358
ericas.csv 16358
fax.csv 16358
ietf80-1st-timers.csv 16358
geopriv.csv 16374
bcause.csv 16374
ehip.csv 16374
kitten.csv 16377
oauth-ext-review.csv 16377
udp35.csv 16377
hash.csv 16377
hls-interest.csv 16377
savi.csv 16412
vwrap.csv 16428
95all.csv 16428
maitai.csv 16428
mpls-tp.csv 16435
codec.csv 16436
mip6.csv 16487
add.csv 16487
json-canon.csv 16487
bgp-autoconf.csv 16488
ldapbis.csv 16503
iporpr.csv 16526
102-newcomers.csv 16526
curdle.csv 16532
cso.csv 16532
pesci-discuss.csv 16532
sica.csv 16532
ancp.csv 16550
ietf-mentees.csv 16550
87all.

/Users/pkhare/anaconda3/envs/py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


ietf.csv 18778
openpgp.csv 18975
sming.csv 18975
snmpconf.csv 19004
rift.csv 19004
webtransport.csv 19004
ietf79-1st-timers.csv 19004
cip.csv 19004
97companions.csv 19004
arp222.csv 19004
antitrust-policy.csv 19004
mentoring-coordinators.csv 19006
ipv6.csv 19027
sacred.csv 19045
pem.csv 19045
sipping-tispan.csv 19045
tnfs.csv 19045
messaging.csv 19045
time.csv 19045
wrec.csv 21697
spam.csv 21979
rfc-interest.csv 21990
76all.csv 21990
eapext.csv 21990
cwt-reg-review.csv 21990
6tsch.csv 21990
ioam.csv 21990
hasmat.csv 21990
media-types.csv 21991
ltans.csv 21992
jose.csv 22125


In [8]:
pid_rolebased = set([])
for eid in emailID_pid_dict:
    if eid in role_based_emailIDs:
        #print(eid)
        pid_rolebased.add(int(emailID_pid_dict[eid]))
print(len(pid_rolebased))

77


In [25]:
#yearly_bwcnt_nodes_full = dict()
#yearly_egnveccnt_nodes_full_1yr_window = dict()
yearly_egnveccnt_nodes_full = dict()

lst_spam_messageIDs = list(spam_messageIDs)
for yr in range(1999,2021):#[2004,2009,2014,2019]:#[2000,2005,2010,2015,2019]:#range(2000, 2021):
    for typ in ['overall']:#'wg'#['wg','rg','meeting','overall']:#,'dir','review','announcements']:
        
        if typ != 'overall':
            #df_mappings4 = df_mappings.loc[(df_mappings.Type == 'reply_to')  & (df_mappings.Interaction_timestamp <= date(yr,12,31)) & (df_mappings.Mailinglist_type == typ) & (~df_mappings.MessageID_A.isin(lst_spam_messageIDs)) & (~df_mappings.MessageID_B.isin(lst_spam_messageIDs))]
            #LOOKING FOR LAST 5 YEAR HISTORY
            df_mappings4 = df_mappings.loc[(df_mappings.Type == 'reply_to')  & (df_mappings.Interaction_timestamp <= date(yr,12,31))  & (df_mappings.Interaction_timestamp > date(yr-5,12,31)) & (df_mappings.Mailinglist_type == typ) & (~df_mappings.MessageID_A.isin(lst_spam_messageIDs)) & (~df_mappings.MessageID_B.isin(lst_spam_messageIDs)) & (df_mappings.Mailinglist_type == "wg")]
            #LOOKING FOR 1 YEAR HISTORY
            #df_mappings4 = df_mappings.loc[(df_mappings.Type == 'reply_to')  & (df_mappings.Interaction_timestamp <= date(yr,12,31))  & (df_mappings.Interaction_timestamp > date(yr-1,12,31)) & (df_mappings.Mailinglist_type == typ) & (~df_mappings.MessageID_A.isin(lst_spam_messageIDs)) & (~df_mappings.MessageID_B.isin(lst_spam_messageIDs)) & (df_mappings.Mailinglist_type == "wg")]
        else:
            #df_mappings4 = df_mappings.loc[(df_mappings.Type == 'reply_to') & (df_mappings.Interaction_timestamp <= date(yr,12,31)) & (~df_mappings.MessageID_A.isin(lst_spam_messageIDs)) & (~df_mappings.MessageID_B.isin(lst_spam_messageIDs))]
            #LOOKING FOR LAST 5 YEAR HISTORY
            df_mappings4 = df_mappings.loc[(df_mappings.Type == 'reply_to') & (df_mappings.Interaction_timestamp <= date(yr,12,31)) & (df_mappings.Interaction_timestamp > date(yr-5,12,31)) & (~df_mappings.MessageID_A.isin(lst_spam_messageIDs)) & (~df_mappings.MessageID_B.isin(lst_spam_messageIDs))  & (df_mappings.Mailinglist_type == "wg")]
            #LOOKING FOR LAST 1 YEAR HISTORY
            #df_mappings4 = df_mappings.loc[(df_mappings.Type == 'reply_to') & (df_mappings.Interaction_timestamp <= date(yr,12,31)) & (df_mappings.Interaction_timestamp > date(yr-1,12,31)) & (~df_mappings.MessageID_A.isin(lst_spam_messageIDs)) & (~df_mappings.MessageID_B.isin(lst_spam_messageIDs))  & (df_mappings.Mailinglist_type == "wg")]
        
        G1 = nx.Graph()
        
        for i, r in df_mappings4.iterrows():
            
            if r['A_PersonID_From'] in pid_rolebased:
                continue
            if r['B_PersonID_To'] in pid_rolebased:
                continue
            #OPTIONAL   
            #if not (r['Max_Time_A'] - r['Time_since_1st_mail_A'] + r['Interaction_timestamp'].year) >= yr:
            #    continue
            #if not (r['Max_Time_B'] - r['Time_since_1st_mail_B'] + r['Interaction_timestamp'].year) >= yr:
            #    continue
                
            if r['A_PersonID_From'] not in G1:
                G1.add_nodes_from([r['A_PersonID_From']])
            if r['B_PersonID_To'] not in G1:
                G1.add_nodes_from([r['B_PersonID_To']])
                
            G1.add_edges_from([(r['A_PersonID_From'], r['B_PersonID_To'])])
            
        if len(G1) > 0:
          
            d = centrality.eigenvector_centrality(G1)
            d = dict(sorted(d.items(), key=lambda x: x[1], reverse=True))
            
            #yearly_egnveccnt_nodes_full_1yr_window[yr] = d
            yearly_egnveccnt_nodes_full[yr] = d
        #print(yr,len(yearly_egnveccnt_nodes_full_1yr_window),len(G1))
        print(yr,len(yearly_egnveccnt_nodes_full),len(G1))
        G1.clear()

1999 1 1059
2000 2 1578
2001 3 2361
2002 4 3469
2003 5 4513
2004 6 5289
2005 7 6016
2006 8 6514
2007 9 6668
2008 10 6537
2009 11 6617
2010 12 6650
2011 13 6613
2012 14 6571
2013 15 6687
2014 16 6661
2015 17 6529
2016 18 6299
2017 19 6066
2018 20 5701
2019 21 5366
2020 22 5079


In [78]:
#len(yearly_egnveccnt_nodes_full[2000])

In [26]:
pid_percentile = dict()
#yr = 2019
for yr in range(1999,2021):
    #for pid in yearly_PIDs:
        #d = yearly_bwcnt_nodes_full[yr]
    d = yearly_egnveccnt_nodes_full[yr]
    ordered_nodes = []

    for k in d:
        ordered_nodes.append(k)

    #if pid not in ordered_nodes:
    #    continue
    
    for pid in ordered_nodes:

        percentile_position = round((ordered_nodes.index(pid)+1)*100/len(ordered_nodes))
        if yr not in pid_percentile:
            pid_percentile[yr] = {pid : percentile_position}
        else:
            pid_percentile[yr][pid] = percentile_position

    #d2 = yearly_egnveccnt_nodes_full[yr]
    #ordered_nodes2 = []

    #for k in d2:
    #    ordered_nodes2.append(k)

    #if pid not in ordered_nodes2:
    #    continue

    #percentile_position2 = round((ordered_nodes2.index(pid)+1)*100/len(ordered_nodes2))

    #avg_percentile = (percentile_position+percentile_position2)/2

    #pid_percentile[pid] = avg_percentile

In [80]:
#pid_percentile[yr][pid]

To generate maillist_yearly_monthly_type_active_status.json (used below) please refer to https://github.com/sodestream/imc2021-submission/blob/main/scripts/emails/generate_maillist_yearly_monthly_type_active_status.py and https://github.com/sodestream/imc2021-submission

In [18]:
maillist_type = dict()
f = open('../../analysis_ietf2/analysis_revision3/maillist_yearly_monthly_type_active_status.json')
maillist_yearly_monthly_type_active_status = json.load(f)
c = 0
for grp in maillist_yearly_monthly_type_active_status:
    flag = True
    c += 1
    for year in maillist_yearly_monthly_type_active_status[grp]:
        if flag:
            for month in maillist_yearly_monthly_type_active_status[grp][year]:
                if flag:
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'wg':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'wg'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'rg':
                        #list_RG.append(grp)
                        maillist_type[grp] = 'rg'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'meeting':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'meeting'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'dir':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'dir'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'review':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'review'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'announcements':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'announcements'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'iab':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'iab'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'rag':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'rag'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'ag':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'ag'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'team':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'team'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'ietf@ietf.org':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'ietf@ietf.org'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'program':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'program'
                        flag = False
                    if maillist_yearly_monthly_type_active_status[grp][year][month]['type'] == 'nomcom':
                        #list_WG.append(grp)
                        maillist_type[grp] = 'nomcom'
                        flag = False
    
                else:
                    break
        else:
            break
            
    if flag:
        maillist_type[grp] = 'other'
        flag = False
print(c, len(maillist_type))


1085 1085


In [12]:
#download from the repository - https://github.com/chbrown/liwc-python
import liwc

The below LIWC dictionary is with postdoc team of Sodestream at QMUL

In [174]:
#use a LIWC dictionary (to be acquired from liwc.net for research purpose)
parse, category_names = liwc.load_token_parser('../LIWC2015_Dictionary.dic')

In [14]:
def tokenize_t(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)

In [15]:
from collections import Counter

For email segmentation python file please contact Mladen Karan

In [16]:
# run email_segmentation.py
%run -i '../analysis_5/email_segmentation.py'

*********************** Example 1 extracting unquoted parts from raw text  *********************************************************
Segments:
*** TYPE: quote***
> This is
> A quote
*** TYPE: normal***
And this is the reply.
*** TYPE: signature***
---
Henry Jones Junior, PhD



 AFTER REMOVING QUOTES:
And this is the reply.




*********************** Example 2 extracting unquoted parts from  EmailMessage class (old lib version)*********************************************************
Segments:
*** TYPE: normal***
Section 3.10, event 16 under DTMF events.

*** TYPE: quote***
Romel Khan wrote:
> How is it possible to pass the hook-flash over RTP?
> I did not seem to find mention of that in RFC2833 under line events.
>
> Thanks.
>
> Romel Khan
> Ph: (732)-363-0213, x241
>
> _______________________________________________
> Audio/Video Transport Working Group
> avt@ietf.org
> http://www.ietf.org/mailman/listinfo/avt
*** TYPE: signature***
_______________________________________________
Au

In [17]:
archive = ma.MailArchive()

generate msgid_messages_d dictionary that contains clean text after removing quotes

In [19]:
#msgid_messages_d = dict()
#for mailing_list_name in archive.mailing_list_names():
## running only for working group (wg) type mailing list 
#    if mailing_list_name not in maillist_type or maillist_type[mailing_list_name] != 'wg':
#        continue
#    print(mailing_list_name)
#    t_d = get_unquoted_texts(archive, mailing_list_name)
#    for key in t_d:
#        msgid_messages_d[key] = t_d[key]

In [20]:
#json.dump(msgid_messages_d, open( "../analysis_5/msgid_messages_wg_d.json", 'w' ) )

temporarily already loaded from a pre-curated dictionary

In [21]:
msgid_messages_d = json.load(open('../analysis_5/msgid_messages_wg_d.json'))

In [22]:
len(msgid_messages_d)

1189236

In [28]:
domain_jargon = set()
with open('../analysis_5/internet_security_jargon.txt') as f:
    Lines = f.readlines()
    for line in Lines:
        line = line.strip()
        for t in line.lower().split():
            domain_jargon.add(t.lower())
        #print(line)
print(len(domain_jargon))

1336


In [30]:
abbr_set = set([])
c = 0
html_doc = "../analysis_5/abbreviation_html.html"
with open(html_doc) as f:
    #read File
    content = f.read()

    soup = BeautifulSoup(content, 'html.parser')
    #print(soup.title)
    for dobj0 in soup.find_all('div', attrs={'class':'Box mt-3 position-relative'}):
        print('table')
        for dobj1 in dobj0.find_all('tbody'):
            #print('body')
            for dobj2 in dobj1.find_all('tr'):
                for dobj3 in dobj2.find('td'):
                    try:
                        #print(dobj3.text)
                        abbr_set.add(dobj3.text.lower().strip())
                    except:
                        #print(dobj3)
                        c += 1
                        continue
print(c, len(abbr_set))


table
12 1506


In [32]:
ambiguous_tech_words = set(['urns','alive','kill','cryptographically','cryptographic','crypto','violating','violates','violate','violation','attacks', 'warning', 'parties', 'party', 'troubleshoot', 'troubleshooting', 'arguments','loss', 'lost', 'attacker', 'secure', 'insecure', 'vulnerability', 'vulnerable', 'threat', 'threats', 'attack', 'kill', 'argument', 'abuse', 'contradict', 'contradicts', 'loss', 'sender', 'receiver', 'receives', 'version', 'versions', 'attributes', 'attribute', 'identifier', 'identifiers', 'video', 'view', 'views', 'radio', 'telephone', 'click', 'voice', 'speaker', 'sound', 'image', 'sounds', 'loose', 'audio', 'hear', 'search', 'searching', 'fingerprints', 'fingerprint', 'banana', 'arch', 'vanilla', 'architecture', 'operates', 'operate', 'body', 'weight', 'operational', 'operation', 'bodies', 'head', 'heads', 'cookies', 'congestion', 'operator', 'operators', 'diagnostic', 'hand', 'boilerplate', 'colon', 'sleep', 'std', 'std13', 'expression', 'bar', 'parent', 'child','childern', 'request', 'requests', 'compliance', 'routers', 'router','bounce', 'bounces', 'bounced', 'connection', 'connections', 'operational','credential', 'credentials', 'gateway', 'gateways', 'tradeoff', 'killed', 'kill', 'operations'])


In [34]:
catgs = []
for i in category_names:
    catgs.append(i)
len(catgs)

73

# RQ 1

In [35]:
usr_wg_LIWC_perc_d = dict()

messageID_wg_LIWC_catg = dict()

error_count = 0
count = 0
count2 = 1

for yr1 in [2019]:#[2004, 2009, 2014, 2019]:
    print(yr1)
    for mailing_list_name in archive.mailing_list_names():
        if mailing_list_name not in maillist_type or maillist_type[mailing_list_name] != 'wg':
            continue
        #if mailing_list_name in ['opsawg','rtgwg','int-area','tsvwg','dispatch','gendispatch','ops-area','secdispatch','tsv-area']:
        print("\rList- %s , %d , %d" % (mailing_list_name, len(usr_wg_LIWC_perc_d), count2), end = '')
        count2 += 1
        ml = archive.mailing_list(mailing_list_name)

        if ml:
            if ml._num_messages > 0:
                ml_df = ml.messages_dataframe()
                for index, row in ml_df.iterrows():
                    count += 1
                    try:
                        yr = row['Date'].year
                        #if yr < 2015 or yr > 2019:#yr != 2014:
                        #    continue
                        if yr < yr1-4 or yr > yr1:
                            continue

                        if index in spam_messageIDs or index not in msgid_messages_d:
                            continue

                        e = row['From']

                        e = e.replace("'","__apostrophe__")
                        x = re.findall(ren,str(e))

                        if len(x) == 0:
                            x = re.findall(ren2,str(e))
                            if len(x) > 0:
                                email = x[0]
                        else:
                            email = x[0][0]
                        email = email.replace("__apostrophe__","'")#.lower()

                        e = e.replace(email,'')

                        email = email.lower()

                        if '@' not in email:
                            email = email.replace(' at ','@').lower()
                        if email in role_based_emailIDs or email in automated_list or email not in emailID_pid_dict:
                            continue

                        #eid = r['From_emailID']
                        if email in emailID_pid_dict:
                            pid = emailID_pid_dict[email]

                            #if pid not in yearly_PIDs or pid not in pid_percentile:
                            #    continue
                            if pid not in pid_percentile[yr1]:
                                continue
                        else:
                            continue

                        #bdy = msgid_messages_d[message.message_id].lower()
                        body_email = msgid_messages_d[index].lower()
                        body_email = body_email.replace('\n',' ')

                        body_tokens = tokenize_t(body_email)

                        #category_counts = Counter(category for token in body_tokens if token not in domain_jargon and token not in ambiguous_tech_words for category in parse(token))
                        
                        category_counts = dict()
                        
                        for token in body_tokens:
                            if token in domain_jargon or token in abbr_set or token in ambiguous_tech_words:# or any(token in s for s in ambiguous_tech_words) or token in abbr_set:#token in ambiguous_tech_words:
                                continue
                            if len(token) > 3:
                                if any(item.startswith(token) for item in ambiguous_tech_words) or any(token.startswith(item) for item in ambiguous_tech_words):
                                    continue
                                    
                            for category in parse(token):
                                category_counts[category] = category_counts.get(category,0) + 1
                        
                        category_counts = Counter(category_counts)
                            

                        #if pid not in usr_wg_LIWC_perc_d:
                        #    usr_wg_LIWC_perc_d[pid] = {'Email_count':1,'Percentile':pid_percentile[pid]}
                        #else:
                        #    usr_wg_LIWC_perc_d[pid]['Email_count'] = usr_wg_LIWC_perc_d[pid]['Email_count'] + 1

                        if yr1 not in usr_wg_LIWC_perc_d:
                            usr_wg_LIWC_perc_d[yr1] = {pid : {'Email_count':1,'Percentile':pid_percentile[yr1][pid]}}
                        else:
                            if pid not in usr_wg_LIWC_perc_d[yr1]:
                                usr_wg_LIWC_perc_d[yr1][pid] = {'Email_count':1,'Percentile':pid_percentile[yr1][pid]}
                            else:
                                usr_wg_LIWC_perc_d[yr1][pid]['Email_count'] = usr_wg_LIWC_perc_d[yr1][pid]['Email_count'] + 1

                        #messageID_wg_LIWC_catg[index] = {'Percentile':pid_percentile[pid]}
                        messageID_wg_LIWC_catg[index] = {'Percentile':pid_percentile[yr1][pid]}
                        #for catg_ in category_counts:
                        for catg_ in catgs:

                            #usr_wg_LIWC_perc_d[pid][catg_] = usr_wg_LIWC_perc_d[pid].get(catg_,0) + category_counts[catg_]
                            usr_wg_LIWC_perc_d[yr1][pid][catg_] = usr_wg_LIWC_perc_d[yr1][pid].get(catg_,0) + category_counts[catg_]

                            messageID_wg_LIWC_catg[index][catg_] = category_counts[catg_]


                    except Exception as e:
                        print(e)
                        error_count += 1



2019
List- calsify , 1 , 42ames , 1 , 40'NoneType' object has no attribute 'replace'
List- detnet , 1 , 66 631 , 43'NoneType' object has no attribute 'replace'
List- dmm , 1 , 7372 71'NoneType' object has no attribute 'replace'
List- lp-wan , 1 , 17267145 123'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
List- tzdist , 1 , 337333272'NoneType' object has no attribute 'replace'
List- yam , 1 , 361 360, 352

In [36]:
usr_wg_LIWC_perc_df = pd.DataFrame(columns = ['PID']+['Year']+catgs+['Percentile'])

for yr in usr_wg_LIWC_perc_d:

#for p in usr_wg_LIWC_perc_d:

    for p in usr_wg_LIWC_perc_d[yr]:
        lst = []
        lst.append(p)
        lst.append(yr)
        for catg_ in catgs:
            #lst.append(np.log((usr_wg_LIWC_perc_d[p][catg_]+1)/usr_wg_LIWC_perc_d[p]['Email_count']))
            lst.append(np.log((usr_wg_LIWC_perc_d[yr][p][catg_]+1)/usr_wg_LIWC_perc_d[yr][p]['Email_count']))
        #lst.append(usr_wg_LIWC_perc_d[p]['Percentile'])
        lst.append(usr_wg_LIWC_perc_d[yr][p]['Percentile'])
        usr_wg_LIWC_perc_df.loc[0 if pd.isnull(usr_wg_LIWC_perc_df.index.max()) else usr_wg_LIWC_perc_df.index.max() + 1] = lst
        print("\rShape- %d %d" % (usr_wg_LIWC_perc_df.shape[0],usr_wg_LIWC_perc_df.shape[1]), end = '')

Shape- 5363 76

In [38]:
usr_wg_LIWC_perc_df.head()

,PID,Year,function,pronoun,ppron,i,we,you,shehe,they,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Percentile
0,100889.0,2019.0,4.234107,2.765209,1.858135,0.995428,0.648027,0.323787,-1.734601,-1.041454,...,-0.635989,-3.526361,-3.526361,0.211309,-3.526361,-1.916923,-0.390866,-0.887303,-2.427748,18.0
1,101304.0,2019.0,3.295837,1.945910,1.609438,1.609438,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,0.693147,0.000000,0.000000,93.0
2,100149.0,2019.0,3.155662,1.890392,1.114413,0.098885,-0.387192,0.058601,-3.618491,-1.708948,...,-2.114413,-3.826130,-6.391079,-0.865626,-5.292467,-2.135467,-1.633188,-2.411398,-3.500707,2.0
3,101114.0,2019.0,4.161021,2.822585,1.926067,0.879797,0.843926,0.450290,-2.790836,-0.695891,...,-1.206716,-2.403390,-4.736747,0.290008,-3.539043,-0.790322,-0.566213,-1.363006,-2.773137,1.0
4,101226.0,2019.0,3.356594,1.934039,1.070690,0.611654,-0.853112,-0.555277,-4.339467,-2.547708,...,-2.442347,-3.828641,-4.744932,-0.737599,-5.438079,-2.393557,-1.189584,-2.347037,-4.744932,3.0


In [37]:
from sklearn.datasets import make_regression
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr, spearmanr
from sklearn.neural_network import MLPRegressor


In [39]:
#df = pd.read_csv("./liwc_feats.csv")
#df = pd.read_csv("./user_wg_liwc_infl_features.csv")
#df = pd.read_csv("./user_wg_word_infl_features.csv")
#df1 = pd.read_csv("./user_wg_liwc_infl_features_20years.csv")
#df1 = pd.read_csv("./user_wg_WORDLEVEL_"+catgr+"_catg_infl_features_2015-19.csv")
#df1 = pd.read_csv("./startyear_top10year_liwc.csv")
df1 = usr_wg_LIWC_perc_df.copy()
df1 = df1.drop("PID", axis = 1)
print(df1.head())

for yr in [2019]:#[2004, 2009, 2014, 2019]:
    df = df1.loc[df1.Year == yr]
    df1 = df1.drop("Year", axis = 1)
    df = df1
    feat_cols = [c for c in df.columns if c!="Percentile" and c!="Year"]
    lab_cols = [c for c in df.columns if c=="Percentile"]

    print(feat_cols)
    print(lab_cols)
    data = df[feat_cols]
    labels = df[lab_cols]
    labels.columns = ["centrality"]


    print(data.head())
    print(data.shape)
    print()
    print(labels.head())
    print(labels.shape)


    # some scaling and preparation
    scaler = StandardScaler()
    data[data.columns] = scaler.fit_transform(data[data.columns])
    data = sm.add_constant(data)
    print(data.head())

    # statistical approach with statsmodels
    print("Statistical approach: ")
    model = sm.OLS(labels["centrality"], data)
    res = model.fit()
    print(res.summary())
    text_file = open("Output_"+str(yr)+"_1.txt", "w")

    text_file.write(str(res.summary()))
    text_file.close()
    print()
    print()
    

    # machine learning approach with sklearn
    print("Machine learning approach:")
    X_train, X_test, y_train, y_test = train_test_split(data, labels["centrality"], test_size=0.20, random_state=42, shuffle = True)

    model = LinearRegression(fit_intercept = False)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    print( "Pearson correlation on test set is: " + str(pearsonr(list(preds), y_test)))
    print( "Spearman correlation on test set is: " + str(spearmanr(list(preds), y_test)))

    weights = pd.DataFrame()
    weights["feature"] = data.columns
    weights["weight"] = model.coef_
    print(weights)
    print()
    print()
    # nonlinear models just to see if we can get a better prediction
    print("Machine learning with non linear models (might take a little while):")
    model = MLPRegressor(max_iter = 10000)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    print( "Pearson correlation on test set is: " + str(pearsonr(list(preds), y_test)))
    print( "Spearman correlation on test set is: " + str(spearmanr(list(preds), y_test)))

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


     Year  function   pronoun     ppron         i        we       you  \
0  2019.0  4.234107  2.765209  1.858135  0.995428  0.648027  0.323787   
1  2019.0  3.295837  1.945910  1.609438  1.609438  0.000000  0.000000   
2  2019.0  3.155662  1.890392  1.114413  0.098885 -0.387192  0.058601   
3  2019.0  4.161021  2.822585  1.926067  0.879797  0.843926  0.450290   
4  2019.0  3.356594  1.934039  1.070690  0.611654 -0.853112 -0.555277   

      shehe      they     ipron  ...     money     relig     death  informal  \
0 -1.734601 -1.041454  2.238831  ... -0.635989 -3.526361 -3.526361  0.211309   
1  0.000000  0.000000  1.098612  ...  0.000000  0.000000  0.000000  0.693147   
2 -3.618491 -1.708948  1.272328  ... -2.114413 -3.826130 -6.391079 -0.865626   
3 -2.790836 -0.695891  2.296374  ... -1.206716 -2.403390 -4.736747  0.290008   
4 -4.339467 -2.547708  1.381937  ... -2.442347 -3.828641 -4.744932 -0.737599   

      swear  netspeak    assent    nonflu    filler  Percentile  
0 -3.526361 -1

<ipython-input-39-84dec707162c>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data.columns] = scaler.fit_transform(data[data.columns])
/Users/pkhare/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/pkhare/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

                            OLS Regression Results                            
Dep. Variable:             centrality   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.718
Method:                 Least Squares   F-statistic:                     188.1
Date:                Wed, 01 Feb 2023   Prob (F-statistic):               0.00
Time:                        22:19:01   Log-Likelihood:                -22211.
No. Observations:                5363   AIC:                         4.457e+04
Df Residuals:                    5289   BIC:                         4.506e+04
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           49.9862      0.209    238.873   

Pearson correlation on test set is: (0.8269278434017171, 4.9177303510146145e-270)
Spearman correlation on test set is: SpearmanrResult(correlation=0.810288168802157, pvalue=8.347437362191481e-251)


In [40]:
#manually transform the feature correlation above to a CSV format - output_liwc_features_2019.csv

In [41]:
usr_liwc_features_2019 = pd.read_csv('../analysis_5/output_liwc_features_2019.csv',delimiter=',',header=0)

In [42]:
usr_liwc_features_2019.shape

(74, 7)

In [43]:
usr_liwc_features_2019.loc[(usr_liwc_features_2019['P>|t|'] <= 0.05) & (usr_liwc_features_2019.coef < 0)].feature.tolist()


['we', 'they', 'adj', 'negemo', 'anger', 'bio', 'risk', 'informal']

In [44]:
usr_liwc_features_2019.loc[(usr_liwc_features_2019['P>|t|'] <= 0.05) & (usr_liwc_features_2019.coef > 0)].feature.tolist()


['const',
 'article',
 'conj',
 'number',
 'anx',
 'sad',
 'female',
 'body',
 'health',
 'sexual',
 'ingest',
 'affiliation',
 'reward',
 'space',
 'time',
 'death',
 'swear',
 'netspeak']

# RQ 2

wg_chair_history.csv is gathered from conext2021 repository (paper wasn't submitted). Please contact Stephen McQuistin

In [175]:
wg_chair_history = pd.read_csv('../wg_chair_history.csv')
wg_chair_history.head()

,name,group,start,end
0,Kenny Paterson,cfrg,2015-07-12 01:54:22,2020-01-14 14:14:00
1,Alexey Melnikov,cfrg,2015-07-12 01:54:22,None
2,Nick Sullivan,cfrg,2019-05-01 15:05:02,None
3,Stanislav Smyshlyaev,cfrg,2020-01-14 14:14:00,None
4,Joerg Ott,dtnrg,2015-01-23 04:06:57,2016-04-05 06:40:00


In [81]:
len(wg_chair_history.name.unique())

401

In [48]:
c = 0
for i in wg_chair_history.name.unique():
    #print(i)
    if i.lower() not in name_pid_dict:
        c += 1
        print(i)
print(c)

Niels
NRO Chair
Reddy K
3


only 3 names not in name_pid_dictionary mapping, rest are mapped

In [49]:
wg_chair_start_dates = dict()

for i,row in wg_chair_history.iterrows():
    d = datetime.strptime(row['start'], '%Y-%m-%d %H:%M:%S').date()
    if row['name'].lower() not in wg_chair_start_dates:
        wg_chair_start_dates[row['name'].lower()] =  d
    else:
        #print('repeat record')
        if d < wg_chair_start_dates[row['name'].lower()]:
            wg_chair_start_dates[row['name'].lower()] = d
            #print('update')
        
print(len(wg_chair_start_dates))

401


In [50]:
wg_chair_d = dict()
for i,row in wg_chair_history.iterrows():
    d1 = datetime.strptime(row['start'], '%Y-%m-%d %H:%M:%S').date()
    d2 = 'None'
    if row['end'] != 'None':
        d2 = datetime.strptime(row['end'], '%Y-%m-%d %H:%M:%S').date()
    #print(row['name'],d1,d2)
    if row['group'].lower() not in wg_chair_d:
        wg_chair_d[row['group'].lower()] = {row['name'].lower() : [[d1, d2]]}
    else:
        if row['name'].lower() not in wg_chair_d[row['group'].lower()]:
            wg_chair_d[row['group'].lower()][row['name'].lower()] = [[d1, d2]]
        else:
            wg_chair_d[row['group'].lower()][row['name'].lower()].append([d1,d2])

converting names to PID

In [51]:
c_set = set([])
wg_chair_d2 = dict()
for wg in wg_chair_d:
    for p in wg_chair_d[wg]:
        if p not in name_pid_dict:
            c_set.add(p)
            continue
        if wg not in wg_chair_d2:
            wg_chair_d2[wg] = {name_pid_dict[p] : wg_chair_d[wg][p]}
        else:
            wg_chair_d2[wg][name_pid_dict[p]] = wg_chair_d[wg][p]


In [52]:
wg_chair_d['dtnrg']

{'joerg ott': [[datetime.date(2015, 1, 23), datetime.date(2016, 4, 5)]],
 'kevin fall': [[datetime.date(2015, 1, 23), datetime.date(2016, 4, 5)]]}

In [53]:
wg_chair_d2['dtnrg']

{101048: [[datetime.date(2015, 1, 23), datetime.date(2016, 4, 5)]],
 101140: [[datetime.date(2015, 1, 23), datetime.date(2016, 4, 5)]]}

In [111]:
usr_wg_message_perc_df = pd.DataFrame(columns = ['PID','Year','Percentile','Role','Message','Mailinglist'])

error_count = 0
count = 0
count2 = 1
count3 = 0
for yr1 in [2019]:#[2018]:#[2017]:#[2004, 2009, 2014, 2019]:
    print(yr1)
    for mailing_list_name in archive.mailing_list_names():
        if mailing_list_name not in maillist_type or maillist_type[mailing_list_name] != 'wg':
            continue
        #if mailing_list_name in ['opsawg','rtgwg','int-area','tsvwg','dispatch','gendispatch','ops-area','secdispatch','tsv-area']:
        print("\rList- %s , %d" % (mailing_list_name, usr_wg_message_perc_df.shape[0]), end = '')
        count2 += 1
        ml = archive.mailing_list(mailing_list_name)

        if ml:
            if ml._num_messages > 0:
                ml_df = ml.messages_dataframe()
                for index, row in ml_df.iterrows():
                    count += 1
                    try:
                        yr = row['Date'].year
                        #if yr < 2015 or yr > 2019:#yr != 2014:
                        #    continue
                        #if yr < yr1-4 or yr > yr1:
                        #    continue
                        if yr != yr1:
                            continue

                        if index in spam_messageIDs or index not in msgid_messages_d:
                            continue

                        e = row['From']

                        e = e.replace("'","__apostrophe__")
                        x = re.findall(ren,str(e))

                        if len(x) == 0:
                            x = re.findall(ren2,str(e))
                            if len(x) > 0:
                                email = x[0]
                        else:
                            email = x[0][0]
                        email = email.replace("__apostrophe__","'")#.lower()

                        e = e.replace(email,'')

                        email = email.lower()

                        if '@' not in email:
                            email = email.replace(' at ','@').lower()
                        if email in role_based_emailIDs or email in automated_list or email not in emailID_pid_dict:
                            continue

                        #eid = r['From_emailID']
                        if email in emailID_pid_dict:
                            pid = emailID_pid_dict[email]

                            #if pid not in yearly_PIDs or pid not in pid_percentile:
                            #    continue
                            if pid not in pid_percentile[yr1]:
                                continue
                            #if pid not in pid_percentile_1yr_window[yr1]:
                            #    continue
                        else:
                            continue

                        #bdy = msgid_messages_d[message.message_id].lower()
                        body_email = msgid_messages_d[index].lower()
                        body_email = body_email.replace('\n',' ')
                        body_email = body_email.replace('\t',' ')

                        #body_tokens = tokenize_t(body_email)

                        #category_counts = Counter(category for token in body_tokens if token not in domain_jargon for category in parse(token))

                        #if pid not in usr_wg_LIWC_perc_d:
                        #    usr_wg_LIWC_perc_d[pid] = {'Email_count':1,'Percentile':pid_percentile[pid]}
                        #else:
                        #    usr_wg_LIWC_perc_d[pid]['Email_count'] = usr_wg_LIWC_perc_d[pid]['Email_count'] + 1

                        #role = "None"
                        role = "Non_WGC"

                        if mailing_list_name in wg_chair_d2:
                            if pid in wg_chair_d2[mailing_list_name]:
                                flag = False
                                #row['Date']
                                for dt_range in wg_chair_d2[mailing_list_name][pid]:
                                    if flag == False:#len(wg_chair_d2[mailing_list_name][pid]) > 1 and
                                        if row['Date'] >= dt_range[0]:
                                            if dt_range[1] == "None" or row['Date'] <= dt_range[1]:
                                                #role = "ACT_WGC"
                                                role = "CURRENT_WGC"
                                                flag = True
                                                break
                                            elif row['Date'] > dt_range[1]:
                                                #role = "PREV_WGC"
                                                role = "PRIOR_WGC"

                            else:
                                flag = False
                                for ml in wg_chair_d2:
                                    if flag:
                                        break
                                    if pid in wg_chair_d2[ml]:
                                        for dt_range in wg_chair_d2[ml][pid]:
                                            if flag == False:
                                                if row['Date'] >= dt_range[0]:
                                                    if dt_range[1] == "None" or row['Date'] <= dt_range[1]:
                                                        #role = "OTH_WGC"
                                                        role = "ALLO_WGC"
                                                        flag = True
                                                        break
                                                    elif row['Date'] > dt_range[1]:
                                                        #role = "PREV_WGC"#alt label "PREV_OTH_WGC"
                                                        role = "PRIOR_ALLO_WGC"
                                                        
                        else:
                            flag = False
                            for ml in wg_chair_d2:
                                if flag:
                                    break
                                if pid in wg_chair_d2[ml]:
                                    for dt_range in wg_chair_d2[ml][pid]:
                                        if flag == False:
                                            if row['Date'] >= dt_range[0]:
                                                if dt_range[1] == "None" or row['Date'] <= dt_range[1]:
                                                    #role = "OTH_WGC"
                                                    role = "ALLO_WGC"
                                                    flag = True
                                                    break
                                                elif row['Date'] > dt_range[1]:
                                                    #role = "PREV_WGC"#alt label "PREV_OTH_WGC"
                                                    role = "PRIOR_ALLO_WGC"

                        #lst = [pid,yr1, pid_percentile_1yr_window[yr1][pid], role, body_email]
                        #lst = [pid,yr1, pid_percentile_1yr_window[yr1][pid], role, body_email, mailing_list_name]
                        lst = [pid,yr1, pid_percentile[yr1][pid], role, body_email, mailing_list_name]
                        if role != "Non_WGC": #"None":
                            count3 += 1
                        usr_wg_message_perc_df.loc[0 if pd.isnull(usr_wg_message_perc_df.index.max()) else usr_wg_message_perc_df.index.max() + 1] = lst
                        print("\rShape- %d %d %d" % (usr_wg_message_perc_df.shape[0],usr_wg_message_perc_df.shape[1],count3), end = '')
                        

                    except Exception as e:
                        print(e)
                        error_count += 1



2019
Shape- 52523 6 220084524034716139

In [112]:
#checking last mailing list iterated
print(mailing_list_name)

zeroconf


zerocon is the last mailing list so all the mailing lists are iterated despite time out

In [113]:
usr_wg_message_perc_df.head()

,PID,Year,Percentile,Role,Message,Mailinglist
0,100300,2019,0,ALLO_WGC,"hi tal, hi, i am not a 6lo native, but i rev...",6lo
1,101435,2019,5,ALLO_WGC,"hi suresh, authors, i agree that the ntp time...",6lo
2,101305,2019,5,Non_WGC,i agree with specifying time semantics with mo...,6lo
3,101400,2019,29,Non_WGC,"hi tal, thanks for your comments. the scope ...",6lo
4,101430,2019,48,Non_WGC,[please accept our apologies if you receive mu...,6lo


In [114]:
usr_wg_message_perc_df.Role.unique()

array(['ALLO_WGC', 'Non_WGC', 'PRIOR_ALLO_WGC', 'CURRENT_WGC',
       'PRIOR_WGC'], dtype=object)

In [115]:
usr_wg_role_LIWC_perc_d = dict()
roles_list = usr_wg_message_perc_df.Role.unique()
c = 1
for yr1 in [2019]:
    for pid in usr_wg_message_perc_df.PID.unique():
        print("\r %d , %d" % (c, len(usr_wg_role_LIWC_perc_d)), end = '')
        c += 1
        for role_ in roles_list:
            
            #if role_ == 'None':
            #    role = 1
            #elif role_ == 'ACT_WGC':
            #    role = 0
            #else:
            #    continue
            role = role_
            #if role_ == 'Non_WGC':
            #    role = 1
            #elif role_ == "CURRENT_WGC":
            #    role = 0
            #else:
            #    continue
            
            for i,r, in usr_wg_message_perc_df.loc[(usr_wg_message_perc_df.PID == pid) & (usr_wg_message_perc_df.Year == yr1) & (usr_wg_message_perc_df.Role == role_)].iterrows():

                body_email = r['Message'].lower()#msgid_messages_d[index].lower()
                body_email = body_email.replace('\n',' ')
                body_email = body_email.strip()
                
                if len(body_email) == 0:
                    continue
                    
                #if role_ != 'None':
                #    role = 0
                #else:
                #    role = 1
                

                body_tokens = tokenize_t(body_email)

                #category_counts = Counter(category for token in body_tokens if token not in domain_jargon and token not in ambiguous_tech_words for category in parse(token))

                category_counts = dict()

                for token in body_tokens:
                    if token in domain_jargon or token in abbr_set or token in ambiguous_tech_words:# or any(token in s for s in ambiguous_tech_words) or token in abbr_set:#token in ambiguous_tech_words:
                        continue
                    if len(token) > 3:
                        if any(item.startswith(token) for item in ambiguous_tech_words) or any(token.startswith(item) for item in ambiguous_tech_words):
                            continue

                    for category in parse(token):
                        category_counts[category] = category_counts.get(category,0) + 1

                category_counts = Counter(category_counts)


                #if pid not in usr_wg_LIWC_perc_d:
                #    usr_wg_LIWC_perc_d[pid] = {'Email_count':1,'Percentile':pid_percentile[pid]}
                #else:
                #    usr_wg_LIWC_perc_d[pid]['Email_count'] = usr_wg_LIWC_perc_d[pid]['Email_count'] + 1

                if yr1 not in usr_wg_role_LIWC_perc_d:
                    usr_wg_role_LIWC_perc_d[yr1] = {pid : {role:{'Email_count':1,'Percentile':pid_percentile[yr1][pid]}}}
                else:
                    if pid not in usr_wg_role_LIWC_perc_d[yr1]:
                        usr_wg_role_LIWC_perc_d[yr1][pid] = {role: {'Email_count':1,'Percentile':pid_percentile[yr1][pid]}}
                    else:
                        if role not in usr_wg_role_LIWC_perc_d[yr1][pid]:
                            usr_wg_role_LIWC_perc_d[yr1][pid][role] = {'Email_count':1,'Percentile':pid_percentile[yr1][pid]}
                        else:
                            usr_wg_role_LIWC_perc_d[yr1][pid][role]['Email_count'] = usr_wg_role_LIWC_perc_d[yr1][pid][role]['Email_count'] + 1

                
                #for catg_ in category_counts:
                for catg_ in catgs:

                    usr_wg_role_LIWC_perc_d[yr1][pid][role][catg_] = usr_wg_role_LIWC_perc_d[yr1][pid][role].get(catg_,0) + category_counts[catg_]




 2215 , 1

In [116]:
usr_wg_role_LIWC_perc_df = pd.DataFrame(columns=['PID','Year','Role']+catgs+['Percentile'])

for yr1 in usr_wg_role_LIWC_perc_d:
    
    for pid in usr_wg_role_LIWC_perc_d[yr1]:
        for role in ['CURRENT_WGC','Non_WGC']:
        #for role in ['ACT_WGC','None']:
        #for role in [0,1]:
            if role not in usr_wg_role_LIWC_perc_d[yr1][pid]:
                continue
            #if role == "None":
            if role == "Non_WGC":
                r_ = 1
            #elif role == "ACT_WGC":
            elif role == "CURRENT_WGC":
                r_ = 0
            else:
                continue
            #lst = [pid, yr1, role]
            lst = [pid, yr1, r_]
            
            for catg_ in catgs:
                lst.append(np.log((usr_wg_role_LIWC_perc_d[yr1][pid][role][catg_]+1)/usr_wg_role_LIWC_perc_d[yr1][pid][role]['Email_count']))
                
            lst.append(usr_wg_role_LIWC_perc_d[yr1][pid][role]['Percentile'])
            usr_wg_role_LIWC_perc_df.loc[0 if pd.isnull(usr_wg_role_LIWC_perc_df.index.max()) else usr_wg_role_LIWC_perc_df.index.max() + 1] = lst
            print("\r %d , %d" % (usr_wg_role_LIWC_perc_df.shape[0], usr_wg_role_LIWC_perc_df.shape[1]), end = '')
            
            
    

 2098 , 77

In [117]:
print(usr_wg_role_LIWC_perc_df.loc[usr_wg_role_LIWC_perc_df.Role == 1].shape[0])
print(usr_wg_role_LIWC_perc_df.loc[usr_wg_role_LIWC_perc_df.Role == 0].shape[0])
usr_wg_role_LIWC_perc_df.head()

1946
152


,PID,Year,Role,function,pronoun,ppron,i,we,you,shehe,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Percentile
0,101305.0,2019.0,1.0,3.554934,2.332566,1.726899,0.756326,0.693147,-0.226773,-1.749200,...,-1.526056,-3.540959,-4.234107,-0.123233,-4.234107,-3.540959,-0.208755,-2.624669,-4.234107,5.0
1,101400.0,2019.0,1.0,4.193435,2.833213,2.154665,0.693147,1.321756,1.056053,-2.079442,...,-2.079442,-0.980829,-2.079442,0.559616,-2.079442,-0.287682,-0.287682,-0.693147,-2.079442,29.0
2,101430.0,2019.0,1.0,4.345968,2.382628,1.871802,0.606136,0.606136,1.152680,-1.791759,...,1.427116,-1.791759,-1.791759,0.916291,-1.791759,0.773190,-1.791759,-0.693147,-1.791759,48.0
3,101406.0,2019.0,1.0,4.221511,2.819754,2.002708,1.483669,0.342945,0.167054,-3.091042,...,-1.704748,-0.788457,-3.091042,-0.526093,-3.091042,-1.299283,-1.299283,-2.397895,-2.397895,17.0
4,100140.0,2019.0,1.0,3.903905,2.597158,1.856963,1.303144,0.210295,0.021053,-3.157000,...,-2.058388,-3.850148,-2.751535,0.081678,-3.850148,-0.323787,-0.854415,-2.058388,-3.157000,6.0


In [118]:
#can rename ACT_WGC_vs_None_LIWC_scores.csv to CURRENT_WGC_vs_Non_WGC_LIWC_scores.csv
#usr_wg_role_LIWC_perc_df.to_csv('../analysis_5/ACT_WGC_vs_None_LIWC_scores.csv',index=False)

In [119]:
usr_wg_role_LIWC_perc_df.loc[usr_wg_role_LIWC_perc_df.Role==0].shape[0]*100/usr_wg_role_LIWC_perc_df.loc[usr_wg_role_LIWC_perc_df.Role==1].shape[0]


7.810894141829394

In [123]:
feat_cols = [c for c in usr_wg_role_LIWC_perc_df.columns if c!="Percentile" and c!='PID' and c!='Year' and c!='Role']
lab_cols = [c for c in usr_wg_role_LIWC_perc_df.columns if c=="Role"]

temp_df = usr_wg_role_LIWC_perc_df.copy()
print(temp_df.shape[0])
temp_df = temp_df.drop(temp_df.query('Role == 1').sample(frac=.92, random_state=42).index)
print(temp_df.shape[0])
print(feat_cols)
print(lab_cols)
data = temp_df[feat_cols]
labels = temp_df[lab_cols]


2098
308
['function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achiev', 'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent', 'nonflu', 'filler']
['Role']


In [124]:
temp_df.loc[temp_df.Role==0].shape[0]*100/temp_df.loc[temp_df.Role==1].shape[0]


97.43589743589743

In [125]:
print(temp_df.loc[temp_df.Role==0].shape[0])
print(temp_df.loc[temp_df.Role==1].shape[0])

152
156


In [126]:
data.head()

,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,...,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler
5,4.078445,2.549724,1.779783,0.538997,0.517943,0.656780,-2.233592,-0.559616,1.925291,0.000000,...,-4.025352,-0.767255,-4.025352,-4.025352,0.331357,-2.639057,-1.192138,-1.540445,-0.093526,-3.332205
7,2.614960,1.540445,0.510826,-0.405465,-0.405465,0.000000,-1.098612,-1.098612,1.203973,-1.098612,...,-0.405465,-1.098612,-1.098612,-1.098612,-0.405465,-1.098612,-1.098612,-1.098612,-0.405465,-1.098612
8,2.197225,0.916291,0.405465,-0.693147,-0.693147,0.405465,-0.693147,-0.693147,0.405465,-0.693147,...,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147
16,3.877727,2.506886,1.639459,0.948158,0.133531,0.009479,-3.555348,-0.847298,1.962105,-0.495077,...,0.000000,-2.456736,-4.653960,-3.960813,-0.028988,-3.960813,-0.990399,-0.825319,-1.945910,-3.044522
17,3.321645,1.991092,1.443453,-0.125163,0.000000,0.750306,-3.526361,-1.128465,1.137079,-0.818310,...,-2.833213,-2.427748,-3.526361,-3.526361,-1.329136,-3.526361,-2.140066,-2.140066,-3.526361,-2.427748


In [127]:
log_reg = sm.Logit(labels, data).fit(maxiter = 100)#maxiter = 1000000

Optimization terminated successfully.
         Current function value: 0.263625
         Iterations 9


In [128]:
print(log_reg.summary())
#text_file = open("../analysis_5/Output_ACTWGC_vs_None.txt", "w")
#text_file.write(str(log_reg.summary()))
#text_file.close()

                           Logit Regression Results                           
Dep. Variable:                   Role   No. Observations:                  308
Model:                          Logit   Df Residuals:                      235
Method:                           MLE   Df Model:                           72
Date:                Thu, 02 Feb 2023   Pseudo R-squ.:                  0.6196
Time:                        18:07:37   Log-Likelihood:                -81.197
converged:                       True   LL-Null:                       -213.46
Covariance Type:            nonrobust   LLR p-value:                 8.428e-24
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
function         3.5065      2.135      1.642      0.101      -0.678       7.691
pronoun         -3.3153      4.030     -0.823      0.411     -11.213       4.582
ppron            4.9085      2.842      1.72

# RQ 3

In [131]:
pid_percentile_years = dict()

#for pid in pid_emailID_dict:#top_10_2019:

for yr in range(1999,2021):

    d = yearly_egnveccnt_nodes_full[yr]
    #ordered_nodes = []

    #for k_ in d:
    #    ordered_nodes.append(k_)

    #if pid not in ordered_nodes:
    #    continue
    c = 1
    for pid in d:
        percentile_position = round((c*100)/len(d))
        c += 1

    #percentile_position = round((ordered_nodes.index(pid)+1)*100/len(ordered_nodes))
        k = percentile_position
        if k/10 <= 1:
            k_range = 0
        elif k/10 <= 2:
            k_range = 1
        elif k/10 <= 3:
            k_range = 2
        elif k/10 <= 4:
            k_range = 3
        elif k/10 <= 5:
            k_range = 4
        elif k/10 <= 6:
            k_range = 5
        elif k/10 <= 7:
            k_range = 6
        elif k/10 <= 8:
            k_range = 7
        elif k/10 <= 9:
            k_range = 8
        elif k/10 <= 10:
            k_range = 9

        if pid not in pid_percentile_years:
            pid_percentile_years[pid] = {k_range : [yr]}
        else:
            if k_range in pid_percentile_years[pid]:
                pid_percentile_years[pid][k_range].append(yr)
            else:
                pid_percentile_years[pid][k_range] = [yr]
    print("\rDone- %d " % (yr), end = '')

Done- 2020 

In [132]:
pid_50_plus = dict()

for pid in pid_percentile_years:
    if len(set([5,6,7,8,9]).intersection(set([k for k in pid_percentile_years[pid]]))) > 0:
        pid_50_plus[pid] = pid_percentile_years[pid]

In [133]:
print(len(pid_50_plus), len(pid_percentile_years))

14534 18139


In [134]:
pid_50_plus2 = dict()
for pid in pid_50_plus:
    for i in [0,1,2,3,4,5,6,7,8,9]:
        if i in pid_50_plus[pid]:
            if pid not in pid_50_plus2:
                pid_50_plus2[pid] = {i : min(pid_50_plus[pid][i])}
            else:
                pid_50_plus2[pid][i] = min(pid_50_plus[pid][i])

In [136]:
pid_liwc_changes_percntl = dict()
error_count = 0
count2 = 0

#r_temp_list = set(["lol","death","wealth","love","great","uncle"])

for mailing_list_name in archive.mailing_list_names():
    if mailing_list_name not in maillist_type or maillist_type[mailing_list_name] != 'wg':
        continue
    #if mailing_list_name in ['opsawg','rtgwg','int-area','tsvwg','dispatch','gendispatch','ops-area','secdispatch','tsv-area']:
    print("\rList- %s , %d, %d " % (mailing_list_name, count2, error_count), end = '')
    count2 += 1
    ml = archive.mailing_list(mailing_list_name)

    if ml:
        if ml._num_messages > 0:
            ml_df = ml.messages_dataframe()
            for index, row in ml_df.iterrows():
                count += 1
                try:
                    yr = row['Date'].year
                    #if yr > 2019 or yr < 2015:#yr != 2014:
                    #    continue
                    if index in spam_messageIDs or index not in msgid_messages_d:
                        continue

                    e = row['From']
                    e = e.replace("'","__apostrophe__")
                    x = re.findall(ren,str(e))

                    if len(x) == 0:
                        x = re.findall(ren2,str(e))
                        if len(x) > 0:
                            email = x[0]
                    else:
                        email = x[0][0]
                        
                    if not email:
                        continue
                    email = email.replace("__apostrophe__","'")#.lower()

                    e = e.replace(email,'')

                    email = email.lower()

                    if '@' not in email:
                        email = email.replace(' at ','@').lower()
                    if email in role_based_emailIDs or email in automated_list or email not in emailID_pid_dict:
                        continue

                    #eid = r['From_emailID']
                    if email in emailID_pid_dict:
                        pid = emailID_pid_dict[email]
                        if pid not in pid_50_plus2 or 0 not in pid_50_plus2[pid]:
                            continue
                        #zeroth_yr = pid_50_plus2[pid][0]#max([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
                        #max_perc_yr = pid_50_plus2[pid][max([k for k in pid_50_plus2[pid]])]#min([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
                        
                        #if max_perc_yr > zeroth_yr:
                        #    continue
                        #if yr > zeroth_yr or yr < (max_perc_yr-4):
                        #    continue
                        
                        
                            
                        zeroth_yr = pid_50_plus2[pid][0]#max([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
                        #max_perc_yr = pid_50_plus2[pid][max([k for k in pid_50_plus2[pid]])]#min([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
                        joining_yr = []
                        for k in pid_50_plus[pid]:
                            joining_yr.extend(pid_50_plus[pid][k])

                        joining_yr = min(joining_yr)
                        
                        if joining_yr > zeroth_yr:
                            continue
                            
                        if yr > zeroth_yr or yr < (joining_yr-4):
                            continue
                            
                        flag = False
                        for k in pid_50_plus[pid]:
                            if joining_yr in pid_50_plus[pid][k]:
                                if k < 5:
                                    flag = True
                                    break
                        if flag:
                            continue

                    else:
                        continue
                            
                    body_email = msgid_messages_d[index].lower()
                    if not body_email:
                        continue
                    body_email = body_email.replace('\n',' ')

                    body_tokens = list(tokenize_t(body_email))
                    
                    
                    category_counts = dict()
                        
                    for token in body_tokens:
                        if token in domain_jargon or token in abbr_set or token in ambiguous_tech_words:# or any(token in s for s in ambiguous_tech_words) or token in abbr_set:#token in ambiguous_tech_words:
                            continue
                        if len(token) > 3:
                            if any(item.startswith(token) for item in ambiguous_tech_words) or any(token.startswith(item) for item in ambiguous_tech_words):
                                continue

                        for category in parse(token):
                            category_counts[category] = category_counts.get(category,0) + 1

                    category_counts = Counter(category_counts)
                    
                    for catg_ in catgs:
                        
                        if pid not in pid_liwc_changes_percntl:
                            pid_liwc_changes_percntl[pid] = {yr : {catg_ : [category_counts[catg_]/len(body_tokens)]}}
                        else:
                            if yr not in pid_liwc_changes_percntl[pid]:
                                pid_liwc_changes_percntl[pid][yr] = {catg_ : [category_counts[catg_]/len(body_tokens)]}
                            else:
                                if catg_ not in pid_liwc_changes_percntl[pid][yr]:
                                    pid_liwc_changes_percntl[pid][yr][catg_] = [category_counts[catg_]/len(body_tokens)]
                                else:
                                    pid_liwc_changes_percntl[pid][yr][catg_].append(category_counts[catg_]/len(body_tokens))
                                    
                    #common_rw = 0
                    #for token in body_tokens:
                    #    if token in r_temp_list:
                    #        common_rw += 1
                    
                    
                    
                                    
                except Exception as e:
                    print(e)#, email, body_email)
                    error_count += 1

List- abfab , 5, 0 'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
List- bmwg , 36, 15  5   15 'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
List- calsify , 41, 17 mes , 39, 17 'NoneType' object has no attribute 'replace'
List- ccamp , 46, 18  , 42, 18 division by zero
List- detnet , 65, 19 19 'NoneType' object has no attribute

In [137]:
catg_pct_df = pd.DataFrame(columns= ['PID']+catgs+['Percentile'])
for pid in pid_liwc_changes_percntl:
    
    yr_0 = pid_50_plus2[pid][0]#max([k for k in pid_liwc_changes_percntl[pid]])#
    yr_least_pctl = pid_50_plus2[pid][max([k for k in pid_50_plus2[pid]])]#min([k for k in pid_liwc_changes_percntl[pid]])#
    
    ##below new part                       
    zeroth_yr = pid_50_plus2[pid][0]#max([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
    #max_perc_yr = pid_50_plus2[pid][max([k for k in pid_50_plus2[pid]])]#min([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
    joining_yr = []
    for k in pid_50_plus[pid]:
        joining_yr.extend(pid_50_plus[pid][k])

    joining_yr = min(joining_yr)

    if joining_yr > zeroth_yr:
        continue

    #if yr > zeroth_yr or yr < (joining_yr-4):
    #    continue

    flag = False
    for k in pid_50_plus[pid]:
        if joining_yr in pid_50_plus[pid][k]:
            if k < 5:
                flag = True
                break
    if flag:
        continue
        
    if yr_0 not in pid_liwc_changes_percntl[pid] or joining_yr not in pid_liwc_changes_percntl[pid]:
        continue
        
    ##above new part
    
    #below line removed for changes between yr_least_pctl and joining_yr
    #if yr_0 not in pid_liwc_changes_percntl[pid] or yr_least_pctl not in pid_liwc_changes_percntl[pid]:
    #    continue
        
    lst = []
    lst.append(pid)
    
    
    for cat_ in catgs:
        
        l1 = []
        
        for y in range(joining_yr - 4,joining_yr + 1):
            if y in pid_liwc_changes_percntl[pid]:
                l1.extend(pid_liwc_changes_percntl[pid][y][cat_])
        lst.append(np.log((np.sum(l1)+1)/len(l1)))
        
        #for y in range(yr_least_pctl - 4,yr_least_pctl + 1):
        #    if y in pid_liwc_changes_percntl[pid]:
        #        l1.extend(pid_liwc_changes_percntl[pid][y][cat_])
        #lst.append(np.log((np.sum(l1)+1)/len(l1)))
        
        ##lst.append(np.log((np.sum(pid_liwc_changes_percntl[pid][yr_least_pctl][cat_])+1)/len(pid_liwc_changes_percntl[pid][yr_least_pctl][cat_])))
        ##for i in pid_liwc_changes_percntl[pid][yr_least_pctl][cat_]:
    
    #for k in pid_50_plus2[pid]:
    #    if pid_50_plus2[pid][k] == yr_least_pctl:
    #        p_least = k#print(k)
    #        break    
    lst.append(1)
    #d = yearly_egnveccnt_nodes_full[yr_least_pctl]
    
    #p_least = round(([k for k in d].index(pid)+1)*100/len(d))
    #lst.append(p_least)
        
    catg_pct_df.loc[0 if pd.isnull(catg_pct_df.index.max()) else catg_pct_df.index.max() + 1] = lst
    print("\rShape- %d %d" % (catg_pct_df.shape[0],catg_pct_df.shape[1]), end = '')
    
    lst = []
    lst.append(pid)
    
    for cat_ in catgs:
        
        l1 = []
        
        for y in range(yr_0 - 4,yr_0 + 1):
            if y in pid_liwc_changes_percntl[pid]:
                l1.extend(pid_liwc_changes_percntl[pid][y][cat_])
        lst.append(np.log((np.sum(l1)+1)/len(l1)))
        
        #lst.append(np.log((np.sum(pid_liwc_changes_percntl[pid][yr_0][cat_])+1)/len(pid_liwc_changes_percntl[pid][yr_0][cat_])))
        #for i in pid_liwc_changes_percntl[pid][yr_least_pctl][cat_]:
        
    lst.append(0)
    #lst.append(-1)
    #d = yearly_egnveccnt_nodes_full[yr_0]
    
    #p_top10 = round(([k for k in d].index(pid)+1)*100/len(d))
    #lst.append(p_top10)
        
    catg_pct_df.loc[0 if pd.isnull(catg_pct_df.index.max()) else catg_pct_df.index.max() + 1] = lst
    print("\rShape- %d %d" % (catg_pct_df.shape[0],catg_pct_df.shape[1]), end = '')
        

Shape- 1110 75

In [176]:
#catg_pct_df.to_csv('../analysis_5/startyear_top10year_liwc_new.csv',sep=',',index=False)

In [139]:
catg_pct_df.head()

,PID,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Percentile
0,100755.0,-0.700376,-1.207475,-1.405489,-1.537992,-1.573941,-1.499650,-1.609438,-1.609438,-1.371587,...,-1.609438,-1.609438,-1.609438,-1.564761,-1.609438,-1.579935,-1.602320,-1.593813,-1.609438,1.0
1,100755.0,-1.158535,-2.254861,-2.710428,-2.979873,-3.097522,-3.117945,-3.295837,-3.285787,-2.584725,...,-3.295837,-3.295837,-3.295837,-3.050790,-3.274307,-3.123985,-3.218723,-3.278826,-3.295837,0.0
2,100785.0,-1.086618,-1.984817,-2.220003,-2.270473,-2.366518,-2.393903,-2.397895,-2.375229,-2.122716,...,-2.285718,-2.397895,-2.397895,-2.313019,-2.397895,-2.387532,-2.348344,-2.370897,-2.397895,1.0
3,100785.0,-1.265752,-2.615516,-3.186570,-3.388317,-4.111678,-4.125301,-4.304065,-4.211184,-3.098692,...,-4.178090,-4.317488,-4.317488,-3.305623,-4.317488,-3.865287,-3.613581,-4.126607,-4.272137,0.0
4,100036.0,-0.952080,-1.731629,-1.948808,-2.185215,-2.373650,-2.303050,-2.476149,-2.451507,-2.138042,...,-2.484907,-2.484907,-2.484907,-2.356929,-2.484907,-2.390950,-2.447596,-2.472443,-2.484907,1.0


In [148]:
feat_cols2 = [c for c in catg_pct_df.columns if c!="Percentile" and c!='PID' and c!='Year' and c!='Role']
lab_cols2 = [c for c in catg_pct_df.columns if c=="Percentile"]

temp_df2 = catg_pct_df.copy()
print(temp_df2.shape[0])
#temp_df = temp_df.drop(temp_df.query('Role == 1').sample(frac=.92, random_state=42).index)
print(temp_df2.shape[0])
print(feat_cols2)
print(lab_cols2)
data2 = temp_df2[feat_cols2]
labels2 = temp_df2[lab_cols2]


1110
1110
['function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achiev', 'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent', 'nonflu', 'filler']
['Percentile']


In [150]:
log_reg = sm.Logit(labels2, data2).fit()

print(log_reg.summary())
#text_file = open("../analysis_5/Output_startyear_top10year_new"+".txt", "w")
#text_file.write(str(log_reg.summary()))
#text_file.close()

Optimization terminated successfully.
         Current function value: 0.254130
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:             Percentile   No. Observations:                 1110
Model:                          Logit   Df Residuals:                     1037
Method:                           MLE   Df Model:                           72
Date:                Thu, 02 Feb 2023   Pseudo R-squ.:                  0.6334
Time:                        22:36:43   Log-Likelihood:                -282.08
converged:                       True   LL-Null:                       -769.39
Covariance Type:            nonrobust   LLR p-value:                2.855e-158
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
function        26.1064      2.354     11.092      0.000      21.493      30.720
pronoun        -47.1842

# Word based analysis

conducting words level correlation analysis for each LIWC category (with 30 most frequent words in each category) to analyse which words (within the LIWC category) are correlated with influence

In [151]:
catg_wg_words_d = dict()
ambiguous_tech_words_tuple = tuple(list(ambiguous_tech_words))
error_count = 0
count = 0
count2 = 1
for yr1 in [2019]:#][2004, 2009, 2014, 2019]:
    print(yr1)
    for mailing_list_name in archive.mailing_list_names():
        if mailing_list_name not in maillist_type or maillist_type[mailing_list_name] != 'wg':
            continue
        #if mailing_list_name in ['opsawg','rtgwg','int-area','tsvwg','dispatch','gendispatch','ops-area','secdispatch','tsv-area']:
        print("\rList- %s , %d , %d" % (mailing_list_name, len(catg_wg_words_d), count2), end = '')
        count2 += 1
        ml = archive.mailing_list(mailing_list_name)

        if ml:
            if ml._num_messages > 0:
                ml_df = ml.messages_dataframe()
                for index, row in ml_df.iterrows():
                    count += 1
                    try:
                        yr = row['Date'].year
                        #if yr > 2019 or yr < 2015:#yr != 2014:
                        #    continue
                        #if yr < yr1-4 or yr > yr1:
                        #    continue
                        if yr != yr1:
                            continue
                            
                        if index in spam_messageIDs or index not in msgid_messages_d:
                            continue

                        e = row['From']
                        e = e.replace("'","__apostrophe__")
                        x = re.findall(ren,str(e))

                        if len(x) == 0:
                            x = re.findall(ren2,str(e))
                            if len(x) > 0:
                                email = x[0]
                        else:
                            email = x[0][0]
                        email = email.replace("__apostrophe__","'")#.lower()

                        e = e.replace(email,'')

                        email = email.lower()

                        if '@' not in email:
                            email = email.replace(' at ','@').lower()
                        if email in role_based_emailIDs or email in automated_list or email not in emailID_pid_dict:
                            continue

                        
                        body_email = msgid_messages_d[index].lower()
                        body_email = body_email.replace('\n',' ')

                        body_tokens = tokenize_t(body_email)

                        for token in body_tokens:
                            if token in domain_jargon or token in abbr_set or token in ambiguous_tech_words:# or any(token in s for s in ambiguous_tech_words) or token in abbr_set:#token in ambiguous_tech_words:
                                continue
                            if len(token) > 3:
                                if any(item.startswith(token) for item in ambiguous_tech_words) or any(token.startswith(item) for item in ambiguous_tech_words):
                                    continue
                            
                            for category in parse(token):
                                if yr1 not in catg_wg_words_d:
                                    catg_wg_words_d[yr1] = {category : {token : 1}}
                                else:
                                    if category in catg_wg_words_d[yr1]:
                                        catg_wg_words_d[yr1][category][token] = catg_wg_words_d[yr1][category].get(token,0) + 1
                                    else:
                                        catg_wg_words_d[yr1][category] = {token : 1}

                                #if category in catg_wg_words_d:
                                #    catg_wg_words_d[category][token] = catg_wg_words_d[category].get(token,0) + 1
                                #else:
                                #    catg_wg_words_d[category] = {token : 1}

                        #category_counts = Counter(category for token in body_tokens for category in parse(token))


                    except Exception as e:
                        print(e)
                        error_count += 1



2019
List- yam , 1 , 361 360, 352123, 40

In [152]:
usr_wg_WORDLEVEL_perc_d = dict()

catg_obs = 'informal'
#catg_obs = 'body'
error_count = 0
count = 0
count2 = 1

#for c in catg_wg_words_d[2019]:
liwc_catg_word_list = set([])
a = 0
for k in dict(sorted(catg_wg_words_d[2019][catg_obs].items(), key=lambda x: x[1], reverse=True)):
    if a > 29:
        break
    #if k.startswith("milford"):
    #    continue
        
    liwc_catg_word_list.add(k.lower())
    a += 1

for yr1 in [2019]:#[2004, 2009, 2014, 2019]:
    print(yr1)
    for mailing_list_name in archive.mailing_list_names():
        if mailing_list_name not in maillist_type or maillist_type[mailing_list_name] != 'wg':
            continue
        #if mailing_list_name in ['opsawg','rtgwg','int-area','tsvwg','dispatch','gendispatch','ops-area','secdispatch','tsv-area']:
        print("\rList- %s , %d , %d" % (mailing_list_name, len(usr_wg_WORDLEVEL_perc_d), count2), end = '')
        count2 += 1
        ml = archive.mailing_list(mailing_list_name)

        if ml:
            if ml._num_messages > 0:
                ml_df = ml.messages_dataframe()
                for index, row in ml_df.iterrows():
                    count += 1
                    try:
                        yr = row['Date'].year
                        #if yr < 2015 or yr > 2019:#yr != 2014:
                        #    continue
                        if yr < yr1-4 or yr > yr1:
                            continue

                        if index in spam_messageIDs or index not in msgid_messages_d:
                            continue

                        e = row['From']

                        e = e.replace("'","__apostrophe__")
                        x = re.findall(ren,str(e))

                        if len(x) == 0:
                            x = re.findall(ren2,str(e))
                            if len(x) > 0:
                                email = x[0]
                        else:
                            email = x[0][0]
                        email = email.replace("__apostrophe__","'")#.lower()

                        e = e.replace(email,'')

                        email = email.lower()

                        if '@' not in email:
                            email = email.replace(' at ','@').lower()
                        if email in role_based_emailIDs or email in automated_list or email not in emailID_pid_dict:
                            continue

                        #eid = r['From_emailID']
                        if email in emailID_pid_dict:
                            pid = emailID_pid_dict[email]

                            #if pid not in yearly_PIDs or pid not in pid_percentile:
                            #    continue
                            if pid not in pid_percentile[yr1]:
                                continue
                        else:
                            continue

                        #bdy = msgid_messages_d[message.message_id].lower()
                        body_email = msgid_messages_d[index].lower()
                        body_email = body_email.replace('\n',' ')

                        body_tokens = tokenize_t(body_email)
                        
                        if yr1 not in usr_wg_WORDLEVEL_perc_d:
                            usr_wg_WORDLEVEL_perc_d[yr1] = {pid : {'Email_count':1,'Percentile':pid_percentile[yr1][pid]}}
                        else:
                            if pid not in usr_wg_WORDLEVEL_perc_d[yr1]:
                                usr_wg_WORDLEVEL_perc_d[yr1][pid] = {'Email_count':1,'Percentile':pid_percentile[yr1][pid]}
                            else:
                                usr_wg_WORDLEVEL_perc_d[yr1][pid]['Email_count'] = usr_wg_WORDLEVEL_perc_d[yr1][pid]['Email_count'] + 1
                        
                        word_counts = Counter(body_tokens)
                        
                        for t in liwc_catg_word_list:
                            
                            usr_wg_WORDLEVEL_perc_d[yr1][pid][t] = usr_wg_WORDLEVEL_perc_d[yr1][pid].get(t,0) + word_counts[t]

                    except Exception as e:
                        print(e)
                        error_count += 1



2019
List- calsify , 1 , 42ames , 1 , 40'NoneType' object has no attribute 'replace'
List- detnet , 1 , 66 631 , 43'NoneType' object has no attribute 'replace'
List- dmm , 1 , 7372 71'NoneType' object has no attribute 'replace'
List- lp-wan , 1 , 17267145 123'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
List- tzdist , 1 , 337'NoneType' object has no attribute 'replace'
List- yam , 1 , 361 360, 352

In [153]:
list_words = list(liwc_catg_word_list)

usr_wg_WORDLEVEL_perc_df = pd.DataFrame(columns = ['PID']+['Year']+list_words+['Percentile'])

for yr in usr_wg_WORDLEVEL_perc_d:

#for p in usr_wg_LIWC_perc_d:

    for p in usr_wg_WORDLEVEL_perc_d[yr]:
        lst = []
        lst.append(p)
        lst.append(yr)
        
        for w in list_words:
            lst.append(np.log((usr_wg_WORDLEVEL_perc_d[yr][p][w]+1)/usr_wg_WORDLEVEL_perc_d[yr][p]['Email_count']))
            
        #for catg_ in catgs:
        #    #lst.append(np.log((usr_wg_LIWC_perc_d[p][catg_]+1)/usr_wg_LIWC_perc_d[p]['Email_count']))
        #    lst.append(np.log((usr_wg_LIWC_perc_d[yr][p][catg_]+1)/usr_wg_LIWC_perc_d[yr][p]['Email_count']))
        ##lst.append(usr_wg_LIWC_perc_d[p]['Percentile'])
        
        lst.append(usr_wg_WORDLEVEL_perc_d[yr][p]['Percentile'])
        #lst.append(usr_wg_LIWC_perc_d[yr][p]['Percentile'])
        #usr_wg_LIWC_perc_df.loc[0 if pd.isnull(usr_wg_LIWC_perc_df.index.max()) else usr_wg_LIWC_perc_df.index.max() + 1] = lst
        
        usr_wg_WORDLEVEL_perc_df.loc[0 if pd.isnull(usr_wg_WORDLEVEL_perc_df.index.max()) else usr_wg_WORDLEVEL_perc_df.index.max() + 1] = lst
        
        #print("\rShape- %d %d" % (usr_wg_LIWC_perc_df.shape[0],usr_wg_LIWC_perc_df.shape[1]), end = '')
        print("\rShape- %d %d" % (usr_wg_WORDLEVEL_perc_df.shape[0],usr_wg_WORDLEVEL_perc_df.shape[1]), end = '')
        

Shape- 5363 33

In [154]:
usr_wg_WORDLEVEL_perc_df.head(3)

,PID,Year,okay,yes,anyway,hey,txt,indeed,eh,u,...,zzh,o,absolutely,op,um,app,dis,btw,agree,Percentile
0,100889.0,2019.0,-3.526361,-1.446919,-2.427748,-3.526361,-2.427748,-2.427748,-3.526361,-2.427748,...,-3.526361,-3.526361,-3.526361,-3.526361,-3.526361,-3.526361,-3.526361,-3.526361,-2.427748,18.0
1,101304.0,2019.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,93.0
2,100149.0,2019.0,-6.391079,-2.509515,-3.500707,-7.084226,-4.599320,-3.618491,-3.865351,-4.445169,...,-7.084226,-3.420665,-4.193855,-6.391079,-7.084226,-7.084226,-7.084226,-4.445169,-2.894572,2.0


In [155]:
#save this file for later use in feature analysis or continue using dataframe
#usr_wg_WORDLEVEL_perc_df.to_csv('../analysis_5/user_wg_WORDLEVEL_'+catg_obs+'_catg_infl_features_2015-19.csv',sep=',',index=False)


In [156]:
df_rq3 = usr_wg_WORDLEVEL_perc_df.drop("PID", axis = 1)
df_rq3 = df_rq3.loc[df_rq3.Year == 2019]
#df1 = df1.drop("Year", axis = 1)
#df = df1
feat_cols_rq3 = [c for c in df_rq3.columns if c!="Percentile" and c!="Year"]
lab_cols_rq3 = [c for c in df_rq3.columns if c=="Percentile"]

print(feat_cols_rq3)
print(lab_cols_rq3)
data_rq3 = df_rq3[feat_cols_rq3]
labels_rq3 = df_rq3[lab_cols_rq3]
labels_rq3.columns = ["centrality"]

print(data_rq3.shape)
print()
print(labels_rq3.shape)


# some scaling and preparation
scaler = StandardScaler()
data_rq3[data_rq3.columns] = scaler.fit_transform(data_rq3[data_rq3.columns])
data_rq3 = sm.add_constant(data_rq3)
print(data_rq3.head())

# statistical approach with statsmodels
print("Statistical approach: ")
model = sm.OLS(labels_rq3["centrality"], data_rq3)
res = model.fit()
print(res.summary())

#text_file = open("Output_wordlevel_"+str(catg_obs)+".txt", "w")
#e.g., Output_wordlevel_informal.txt
#text_file.write(str(res.summary()))
#text_file.close()

['okay', 'yes', 'anyway', 'hey', 'txt', 'indeed', 'eh', 'u', 'msg', 'oh', 'hmm', 'yeah', 'ok', 'thx', 'rt', 'da', 'apps', 'cc', 'rp', 'k', 'well', 'zzh', 'o', 'absolutely', 'op', 'um', 'app', 'dis', 'btw', 'agree']
['Percentile']
(5363, 30)

(5363, 1)
   const      okay       yes    anyway       hey       txt    indeed  \
0    1.0 -0.972119 -0.148971 -0.488462 -0.911414 -0.605238 -0.472042   
1    1.0  1.162113  1.232140  1.217971  1.149033  1.177887  1.204837   
2    1.0 -2.705911 -1.163239 -1.242631 -2.990270 -2.200207 -1.294504   
3    1.0 -2.717800 -0.255661 -1.182382 -2.596743 -0.940610 -1.352650   
4    1.0 -2.129134 -1.312120 -2.604385 -2.028428 -1.202446 -2.551319   

         eh         u       msg  ...      well       zzh         o  \
0 -0.887685 -0.349383 -1.057755  ...  0.247507 -0.835723 -1.352233   
1  1.151386  1.151960  1.191555  ...  1.150860  1.137239  1.184793   
2 -1.083701 -1.596975 -2.885033  ... -1.446922 -2.826312 -1.276191   
3 -2.789984 -1.955195 -1.791137  ..

<ipython-input-156-1727abae91dd>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_rq3[data_rq3.columns] = scaler.fit_transform(data_rq3[data_rq3.columns])
/Users/pkhare/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/pkhare/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

                            OLS Regression Results                            
Dep. Variable:             centrality   R-squared:                       0.715
Model:                            OLS   Adj. R-squared:                  0.713
Method:                 Least Squares   F-statistic:                     445.1
Date:                Fri, 03 Feb 2023   Prob (F-statistic):               0.00
Time:                        12:23:28   Log-Likelihood:                -22280.
No. Observations:                5363   AIC:                         4.462e+04
Df Residuals:                    5332   BIC:                         4.483e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         49.9862      0.211    236.744      0.0

<b>FURTEHR STEPS</b>: Repeat above steps by updating <b>catg_obs</b> to different LIWC categories such as  "we", "tentat", "they", "time", "body", "health", "death" and others.

# Emails for individual research questions

to be used in BERT vs LIWC comparison and t-snse projections, conducted by Ravi Shekhar

## RQ 1

In [157]:
temp_usr_per_message_df_dict = {'PID':[],'MessageID':[],'Body':[],'Timestamp':[],'Percentile':[]}

#top10_PID = usr_wg_LIWC_perc_df.loc[(usr_wg_LIWC_perc_df.Year == 2019) & (usr_wg_LIWC_perc_df.Percentile <= 10)].PID.unique()
#plus50_PID = usr_wg_LIWC_perc_df.loc[(usr_wg_LIWC_perc_df.Year == 2019) & (usr_wg_LIWC_perc_df.Percentile >= 10)].PID.unique()

error_count = 0
count = 0
count2 = 1

for yr1 in [2019]:#[2004, 2009, 2014, 2019]:
    print(yr1)
    for mailing_list_name in archive.mailing_list_names():
        if mailing_list_name not in maillist_type or maillist_type[mailing_list_name] != 'wg':
            continue
        #if mailing_list_name in ['opsawg','rtgwg','int-area','tsvwg','dispatch','gendispatch','ops-area','secdispatch','tsv-area']:
        #print("\rList- %s , %d" % (mailing_list_name, temp_usr_per_message_df.shape[0]), end = ' ')
        print("\rList- %s , %d" % (mailing_list_name, len(temp_usr_per_message_df_dict['PID'])), end = ' ')
        
        count2 += 1
        ml = archive.mailing_list(mailing_list_name)

        if ml:
            if ml._num_messages > 0:
                ml_df = ml.messages_dataframe()
                for index, row in ml_df.iterrows():
                    count += 1
                    try:
                        yr = row['Date'].year
                        #if yr < 2015 or yr > 2019:#yr != 2014:
                        #    continue
                        if yr < yr1-4 or yr > yr1:
                            continue

                        if index in spam_messageIDs or index not in msgid_messages_d:
                            continue

                        e = row['From']

                        e = e.replace("'","__apostrophe__")
                        x = re.findall(ren,str(e))

                        if len(x) == 0:
                            x = re.findall(ren2,str(e))
                            if len(x) > 0:
                                email = x[0]
                        else:
                            email = x[0][0]
                        email = email.replace("__apostrophe__","'")#.lower()

                        e = e.replace(email,'')

                        email = email.lower()

                        if '@' not in email:
                            email = email.replace(' at ','@').lower()
                        if email in role_based_emailIDs or email in automated_list or email not in emailID_pid_dict:
                            continue

                        #eid = r['From_emailID']
                        if email in emailID_pid_dict:
                            pid = emailID_pid_dict[email]

                            #if pid not in yearly_PIDs or pid not in pid_percentile:
                            #    continue
                            if pid not in pid_percentile[yr1]:
                                continue
                        else:
                            continue
                            
                        #if pid in top10_PID:
                        #    perc_cat = 0
                        #elif pid in plus50_PID:
                        #    perc_cat = 1
                        #else:
                        #    continue
                        
                        ##Now also generating 50 emails max for each PID in 2015-2019, and using actual percentile
                        ## value instead of two categories
                        
                        #if pid_percentile[yr1][pid] <= 10:
                        #    perc_cat = 0
                        #elif pid_percentile[yr1][pid] >= 50:
                        #    perc_cat = 1
                        #else:
                        #    continue
                            
                        #if temp_usr_per_message_df.loc[temp_usr_per_message_df.PID == pid].shape[0] > 50:#20:
                        #    continue

                        #bdy = msgid_messages_d[message.message_id].lower()
                        body_email = msgid_messages_d[index].lower()
                        body_email = body_email.replace('\n',' ')
                        body_email = body_email.replace('\t',' ')
                        
                        #lst = [pid, index, body_email, row['Date'],perc_cat]#pid_messageID_d[pid][msgid]]
                        
                        #lst = [pid, index, body_email, row['Date'],pid_percentile[yr1][pid]]
                        #['PID','MessageID','Body','Timestamp','Percentile']
                        temp_usr_per_message_df_dict['PID'].append(pid)
                        temp_usr_per_message_df_dict['MessageID'].append(index)
                        temp_usr_per_message_df_dict['Body'].append(body_email)
                        temp_usr_per_message_df_dict['Timestamp'].append(row['Date'])
                        temp_usr_per_message_df_dict['Percentile'].append(pid_percentile[yr1][pid])
                        
                        #temp_usr_per_message_df.loc[0 if pd.isnull(temp_usr_per_message_df.index.max()) else temp_usr_per_message_df.index.max() + 1] = lst
                        

                    except Exception as e:
                        print(e)
                        error_count += 1



2019
List- calsify , 29928 ames , 29657 'NoneType' object has no attribute 'replace'
List- detnet , 47391 08 30799 'NoneType' object has no attribute 'replace'
List- dmm , 56921 6 77 'NoneType' object has no attribute 'replace'
List- lp-wan , 141694 5 30 145 'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
List- tzdist , 285645 'NoneType' object has no attribute 'replace'
List- yam , 300802 455 0046 

In [158]:
print(len(temp_usr_per_message_df_dict['PID']))
print(len(temp_usr_per_message_df_dict['MessageID']))
print(len(temp_usr_per_message_df_dict['Body']))
print(len(temp_usr_per_message_df_dict['Timestamp']))
print(len(temp_usr_per_message_df_dict['Percentile']))

300806
300806
300806
300806
300806


In [159]:
temp_usr_per_message_df = pd.DataFrame(temp_usr_per_message_df_dict)

temp_usr_per_message_df.shape

(300806, 5)

In [160]:
#temp_usr_per_message_df.to_csv('../analysis_5/2015-19_asperinfluence_Totalemails.csv',sep='\t',index=False)

### vocabulary comparison - overall vs 2015-2019

In [161]:
total_vocab = set([])
vocab_2019 = set([])
for mailing_list_name in archive.mailing_list_names():
    if mailing_list_name not in maillist_type or maillist_type[mailing_list_name] != 'wg':
        continue
    #if mailing_list_name in ['opsawg','rtgwg','int-area','tsvwg','dispatch','gendispatch','ops-area','secdispatch','tsv-area']:
    #print("\rList- %s , %d" % (mailing_list_name, temp_usr_per_message_df.shape[0]), end = ' ')
    print("\rList- %s , %d , %d" % (mailing_list_name, len(total_vocab), len(vocab_2019)), end = ' ')
    
    ml = archive.mailing_list(mailing_list_name)

    if ml:
        if ml._num_messages > 0:
            ml_df = ml.messages_dataframe()
            for index, row in ml_df.iterrows():
                count += 1
                try:
                    yr = row['Date'].year
                    #if yr < 2015 or yr > 2019:#yr != 2014:
                    #    continue
                    
                    #if yr < yr1-4 or yr > yr1:
                    #    continue

                    if index in spam_messageIDs or index not in msgid_messages_d:
                        continue

                    e = row['From']

                    e = e.replace("'","__apostrophe__")
                    x = re.findall(ren,str(e))

                    if len(x) == 0:
                        x = re.findall(ren2,str(e))
                        if len(x) > 0:
                            email = x[0]
                    else:
                        email = x[0][0]
                    email = email.replace("__apostrophe__","'")#.lower()

                    e = e.replace(email,'')

                    email = email.lower()

                    if '@' not in email:
                        email = email.replace(' at ','@').lower()
                    if email in role_based_emailIDs or email in automated_list or email not in emailID_pid_dict:
                        continue

                    #eid = r['From_emailID']
                    if email in emailID_pid_dict:
                        pid = emailID_pid_dict[email]

                        #if pid not in yearly_PIDs or pid not in pid_percentile:
                        #    continue
                        #
                        #if pid not in pid_percentile[yr1]:
                        #    continue
                    else:
                        continue
                    
                    body_email = msgid_messages_d[index].lower()
                    body_email = body_email.replace('\n',' ')
                    body_email = body_email.replace('\t',' ')
                    
                    if len(body_email) > 0:
                        
                        body_tokens = tokenize_t(body_email)
                        
                        for token in body_tokens:
                            if token in domain_jargon or token in abbr_set or token in ambiguous_tech_words:
                                continue
                            
                            total_vocab.add(token)
                            if yr >= 2015 and yr <= 2019:#yr1-4 or yr > yr1:
                                vocab_2019.add(token)
                                
                except Exception as e:
                    print(e)
                        


List- abfab , 25445 , 16369 'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
List- bmwg , 101678 , 50932    823 'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
List- calsify , 107193 , 52311 6884 , 51934 'NoneType' object has no attribute 'replace'
List- detnet , 150056 , 70352 99 53417 'NoneType' object has no attribute 'replace'

In [162]:
print(len(total_vocab),len(vocab_2019))

735605 212253


In [163]:
email_length_list = []

for i,r in temp_usr_per_message_df.iterrows():
    if pd.notna(r['Body']):
        email_length_list.append(len(r['Body']))


In [164]:
print(len(email_length_list), np.mean(email_length_list), np.median(email_length_list))

300806 958.7520494936936 499.0


## RQ 2

In [170]:
usr_wg_message_perc_df.head()

,PID,Year,Percentile,Role,Message,Mailinglist
0,100300,2019,0,ALLO_WGC,"hi tal, hi, i am not a 6lo native, but i rev...",6lo
1,101435,2019,5,ALLO_WGC,"hi suresh, authors, i agree that the ntp time...",6lo
2,101305,2019,5,Non_WGC,i agree with specifying time semantics with mo...,6lo
3,101400,2019,29,Non_WGC,"hi tal, thanks for your comments. the scope ...",6lo
4,101430,2019,48,Non_WGC,[please accept our apologies if you receive mu...,6lo


In [171]:
#x_df = pd.DataFrame(columns = ["PID","Year","Percentile","Role","Body"])
x_df_dict = {"PID":[],"Year":[],"Percentile":[],"Role":[],"Body":[]}
for i,r in usr_wg_message_perc_df.loc[usr_wg_message_perc_df.Role.isin(['CURRENT_WGC','Non_WGC'])].iterrows():#(['ACT_WGC','None'])].iterrows():
    if r['Role'] == "CURRENT_WGC":#"ACT_WGC":
        p = 0
    elif r['Role'] == "Non_WGC":#"None":
        p= 1
    else:
        continue
    #lst = [r['PID'],r['Year'],p,r['Role'],r['Message']]
    #x_df.loc[0 if pd.isnull(x_df.index.max()) else x_df.index.max() + 1] = lst
    x_df_dict["PID"].append(r['PID'])
    x_df_dict["Year"].append(r['Year'])
    x_df_dict["Percentile"].append(r['Percentile'])
    x_df_dict["Role"].append(p)
    x_df_dict["Body"].append(r['Message'])

x_df = pd.DataFrame(x_df_dict)
#save the dataframe
#x_df.to_csv('../analysis_5/ACT_WGC_emails_2019.csv',sep='\t',index=False)


In [172]:
x_df.head()

,PID,Year,Percentile,Role,Body
0,101305,2019,5,1,i agree with specifying time semantics with mo...
1,101400,2019,29,1,"hi tal, thanks for your comments. the scope ..."
2,101430,2019,48,1,[please accept our apologies if you receive mu...
3,101406,2019,17,1,"hi authors of ""draft-ietf-6lo-fragment-recover..."
4,101406,2019,17,1,hi pascal the limit of 2k datagram doesn't hu...


## RQ 3

In [165]:
pid_messages_changes_percntl = dict()
error_count = 0
count2 = 0

for mailing_list_name in archive.mailing_list_names():
    if mailing_list_name not in maillist_type or maillist_type[mailing_list_name] != 'wg':
        continue
    #if mailing_list_name in ['opsawg','rtgwg','int-area','tsvwg','dispatch','gendispatch','ops-area','secdispatch','tsv-area']:
    print("\rList- %s , %d, %d " % (mailing_list_name, count2, error_count), end = '')
    count2 += 1
    ml = archive.mailing_list(mailing_list_name)

    if ml:
        if ml._num_messages > 0:
            ml_df = ml.messages_dataframe()
            for index, row in ml_df.iterrows():
                count += 1
                try:
                    yr = row['Date'].year
                    #if yr > 2019 or yr < 2015:#yr != 2014:
                    #    continue
                    if index in spam_messageIDs or index not in msgid_messages_d:
                        continue

                    e = row['From']
                    e = e.replace("'","__apostrophe__")
                    x = re.findall(ren,str(e))

                    if len(x) == 0:
                        x = re.findall(ren2,str(e))
                        if len(x) > 0:
                            email = x[0]
                    else:
                        email = x[0][0]
                        
                    if not email:
                        continue
                    email = email.replace("__apostrophe__","'")#.lower()

                    e = e.replace(email,'')

                    email = email.lower()

                    if '@' not in email:
                        email = email.replace(' at ','@').lower()
                    if email in role_based_emailIDs or email in automated_list or email not in emailID_pid_dict:
                        continue

                    #eid = r['From_emailID']
                    if email in emailID_pid_dict:
                        pid = emailID_pid_dict[email]
                        if pid not in pid_50_plus2 or 0 not in pid_50_plus2[pid]:
                            continue
                                                
                            
                        zeroth_yr = pid_50_plus2[pid][0]#max([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
                        #max_perc_yr = pid_50_plus2[pid][max([k for k in pid_50_plus2[pid]])]#min([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
                        joining_yr = []
                        for k in pid_50_plus[pid]:
                            joining_yr.extend(pid_50_plus[pid][k])

                        joining_yr = min(joining_yr)
                        
                        if joining_yr > zeroth_yr:
                            continue
                            
                        if yr > zeroth_yr or yr < (joining_yr-4):
                            continue
                            
                        flag = False
                        for k in pid_50_plus[pid]:
                            if joining_yr in pid_50_plus[pid][k]:
                                if k < 5:
                                    flag = True
                                    break
                        if flag:
                            continue

                    else:
                        continue
                            
                    body_email = msgid_messages_d[index].lower()
                    if not body_email:
                        continue
                        
                    if pid not in pid_messages_changes_percntl:
                            pid_messages_changes_percntl[pid] = {yr:[index]}#{yr : {catg_ : [category_counts[catg_]/len(body_tokens)]}}
                    else:
                        if yr not in pid_messages_changes_percntl[pid]:
                            pid_messages_changes_percntl[pid][yr] = [index]#{catg_ : [category_counts[catg_]/len(body_tokens)]}
                        else:
                            pid_messages_changes_percntl[pid][yr].append(index)
                            #if catg_ not in pid_liwc_changes_percntl[pid][yr]:
                            #    pid_liwc_changes_percntl[pid][yr][catg_] = [category_counts[catg_]/len(body_tokens)]
                            #else:
                            #    pid_liwc_changes_percntl[pid][yr][catg_].append(category_counts[catg_]/len(body_tokens))

                
                except Exception as e:
                    print(e)#, email, body_email)
                    error_count += 1

List- abfab , 5, 0 'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
List- bmwg , 36, 15  5   15 'NoneType' object has no attribute 'replace'
'NoneType' object has no attribute 'replace'
List- calsify , 41, 17 mes , 39, 17 'NoneType' object has no attribute 'replace'
List- detnet , 65, 18 18 2, 18 'NoneType' object has no attribute 'replace'
List- dmm , 72, 19 9 19 'NoneTy

In [166]:
#catg_pct_df = pd.DataFrame(columns= ['PID']+catgs+['Percentile'])
messages_pct_df_dict = {"PID":[],"Body":[],"Percentile":[]}
for pid in pid_messages_changes_percntl:
    
    yr_0 = pid_50_plus2[pid][0]#max([k for k in pid_liwc_changes_percntl[pid]])#
    yr_least_pctl = pid_50_plus2[pid][max([k for k in pid_50_plus2[pid]])]#min([k for k in pid_liwc_changes_percntl[pid]])#
    
    ##below new part                       
    zeroth_yr = pid_50_plus2[pid][0]#max([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
    #max_perc_yr = pid_50_plus2[pid][max([k for k in pid_50_plus2[pid]])]#min([pid_50_plus[pid][krange] for krange in pid_50_plus[pid]])
    joining_yr = []
    for k in pid_50_plus[pid]:
        joining_yr.extend(pid_50_plus[pid][k])

    joining_yr = min(joining_yr)

    if joining_yr > zeroth_yr:
        continue

    #if yr > zeroth_yr or yr < (joining_yr-4):
    #    continue

    flag = False
    for k in pid_50_plus[pid]:
        if joining_yr in pid_50_plus[pid][k]:
            if k < 5:
                flag = True
                break
    if flag:
        continue
        
    if yr_0 not in pid_messages_changes_percntl[pid] or joining_yr not in pid_messages_changes_percntl[pid]:
        continue
        
    ##above new part
    
    #below line removed for changes between yr_least_pctl and joining_yr
    #if yr_0 not in pid_liwc_changes_percntl[pid] or yr_least_pctl not in pid_liwc_changes_percntl[pid]:
    #    continue
        
    #lst = []
    #lst.append(pid)
    
    for y in range(joining_yr - 4,joining_yr + 1):
        if y in pid_messages_changes_percntl[pid]:
            for msg_index in pid_messages_changes_percntl[pid][y]:
                lst = []
                
                body_email = msgid_messages_d[msg_index].lower()
                if not body_email:
                    continue
                body_email = body_email.replace('\n',' ')
                body_email = body_email.replace('\t',' ')
                
                messages_pct_df_dict['PID'].append(pid)
                messages_pct_df_dict['Body'].append(body_email)
                messages_pct_df_dict['Percentile'].append(1)
                
    for y in range(yr_0 - 4,yr_0 + 1):
        if y in pid_messages_changes_percntl[pid]:
            for msg_index in pid_messages_changes_percntl[pid][y]:
                lst = []
                
                body_email = msgid_messages_d[msg_index].lower()
                if not body_email:
                    continue
                body_email = body_email.replace('\n',' ')
                body_email = body_email.replace('\t',' ')

                messages_pct_df_dict['PID'].append(pid)
                messages_pct_df_dict['Body'].append(body_email)
                messages_pct_df_dict['Percentile'].append(0)
    
    
        
    #catg_pct_df.loc[0 if pd.isnull(catg_pct_df.index.max()) else catg_pct_df.index.max() + 1] = lst
    #print("\rShape- %d %d" % (catg_pct_df.shape[0],catg_pct_df.shape[1]), end = '')
    print("\rShape- %d %d" % (len(messages_pct_df_dict['PID']),len(messages_pct_df_dict['Percentile'])), end = '')
    
    #lst = []
    #lst.append(pid)
    
    #for cat_ in catgs:
        
    #    l1 = []
        
    #    for y in range(yr_0 - 4,yr_0 + 1):
    #        if y in pid_liwc_changes_percntl[pid]:
    #            l1.extend(pid_liwc_changes_percntl[pid][y][cat_])
    #    lst.append(np.log((np.sum(l1)+1)/len(l1)))
        
    #    #lst.append(np.log((np.sum(pid_liwc_changes_percntl[pid][yr_0][cat_])+1)/len(pid_liwc_changes_percntl[pid][yr_0][cat_])))
    #    #for i in pid_liwc_changes_percntl[pid][yr_least_pctl][cat_]:
        
    #lst.append(0)

        
    #catg_pct_df.loc[0 if pd.isnull(catg_pct_df.index.max()) else catg_pct_df.index.max() + 1] = lst
    #print("\rShape- %d %d" % (catg_pct_df.shape[0],catg_pct_df.shape[1]), end = '')
messages_pct_df = pd.DataFrame(messages_pct_df_dict)
print(messages_pct_df.shape)
        

Shape- 74463 74463(74463, 3)


In [167]:
#messages_pct_df.to_csv('../analysis_5/startyear_top10year_emails.csv',sep='\t',index=False)